<a href="https://colab.research.google.com/github/Tamil-palace/-face_recognition/blob/master/Latest_smartapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=d5b2b132ba451ea997ac0b433b76003cca69bfaa463dccc5abafa254419dff9a
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.4.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
xarray-eins

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata'
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")
+
# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 1
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
current_positions = {symbol: None for symbol in SYMBOLS}
ltp_data = {symbol: None for symbol in SYMBOLS}
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol, exchange="NSE"):
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params):
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        # SmartApi returns order ID (string) on success or dict with error
        if isinstance(order, str):
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: {order.get('message', 'Unknown error')}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}")
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.error("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id):
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id):
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            obj.cancelOrder(order_id, details.get("variety"))
            logger.info(f"Cancelled order {order_id}")
            return True
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return None

# --- Main Trading Logic ---
def fixed_target_trade(symbol, entry_price, trade_type, quantity, entry_time):
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_time = time.time()
    try:
        while current_positions[symbol]:
            ltp = ltp_data.get(symbol)
            if ltp is None:
                logger.warning(f"No LTP for {symbol}. Attempting to fetch...")
                for _ in range(5):
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token:
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = ltp
                            logger.debug(f"Fetched LTP for {symbol}: {ltp:.2f}")
                            break
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol}: {str(e)}")
                    time.sleep(0.5)
                if ltp is None:
                    if time.time() - start_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol}. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable"
                    else:
                        ltp = entry_price
                        logger.warning(f"Using entry price for {symbol}: {ltp:.2f}")

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        profit_qty = int(current_qty * profit_percentages[i])
                        if profit_qty == 0:
                            continue
                        exit_order_params = {
                            "variety": "NORMAL",
                            "tradingsymbol": tradingsymbol,
                            "symboltoken": str(token),
                            "transactiontype": exit_transaction,
                            "exchange": "NSE",
                            "ordertype": "MARKET",
                            "producttype": "INTRADAY",
                            "duration": "DAY",
                            "price": "0.00",
                            "quantity": str(profit_qty)
                        }
                        exit_order = place_order(exit_order_params)
                        if exit_order:
                            fees = profit_qty * ltp * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (ltp - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - ltp) * profit_qty
                            net_profit = profit - fees
                            total_net_profit += net_profit
                            current_qty -= profit_qty
                            total_profit_taken += 1
                            exit_details.append({"price": ltp, "quantity": profit_qty})
                            partial_profit_taken[symbol][i] = True
                            tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                            logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {ltp:.2f}, Net P/L: {net_profit:.2f}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                        else:
                            logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1))
                }
                order_id = stop_loss_orders[symbol].get("order_id") if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    sl_order_params["orderid"] = order_id
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        try:
                            obj.modifyOrder(sl_order_params)
                            logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}")
                            last_sl_time[symbol] = time.time()
                        except Exception as e:
                            logger.error(f"SL order modification failed for {symbol}: {str(e)}")
                            new_sl_order = place_order(sl_order_params)
                            if new_sl_order:
                                stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                last_sl_time[symbol] = time.time()
                    else:
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()

            if exit_position and current_qty > 0:
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order:
                    fees = current_qty * ltp * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                    profit = (ltp - entry_price) * current_qty if trade_type == "BUY" else (entry_price - ltp) * current_qty
                    net_profit = profit - fees
                    total_net_profit += net_profit
                    exit_details.append({"price": ltp, "quantity": current_qty})
                    with loss_lock:
                        if net_profit < 0:
                            consecutive_losses += 1
                        else:
                            consecutive_losses = 0
                    logger.info(f"Exited {trade_type} for {symbol}: {current_qty} at {ltp:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                current_qty = 0

            if current_qty <= 0 and stop_loss_orders[symbol]:
                order_id = stop_loss_orders[symbol]["order_id"]
                cancel_order(order_id)
                stop_loss_orders[symbol] = None

            if current_qty <= 0 or exit_position:
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": entry_price,
                    "exit_details": exit_details,
                    "net_pnl": total_net_profit,
                    "profit_layers": profit_layers,
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": exit_reason
                }
                current_positions[symbol] = None
                stop_loss_orders[symbol] = None
                partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                break

            time.sleep(1)
    except Exception as e:
        logger.error(f"Fixed target error for {symbol}: {str(e)}")
        current_positions[symbol] = None
        stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol):
    global websocket_active, websocket_instances
    if websocket_active.get(symbol):
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP[symbol]
    correlation_id = f"{symbol}_{int(time.time())}"

    ws = SmartWebSocketV2(auth_token=JWT_TOKEN, api_key=API_KEY, client_code=USERNAME, feed_token=FEED_TOKEN)
    websocket_instances[symbol] = ws

    def on_data(ws, message):
        try:
            logger.debug(f"WebSocket raw message for {symbol}: {message}")
            ltp = message.get('ltp') or (message.get('last_traded_price') / 100 if message.get('last_traded_price') else None)
            if ltp:
                ltp_data[symbol] = float(ltp)
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data error for {symbol}: {str(e)}")

    def on_open(ws):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1,
                "tokenList": [{"exchangeType": 1, "tokens": [int(token)]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}")
        with api_lock:
            global api_failures
            api_failures += 1
            if api_failures >= MAX_API_FAILURES:
                logger.error("Maximum API failures reached. Exiting.")
                sys.exit(1)

    def on_close(ws, *args, **kwargs):
        logger.warning(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs}")
        websocket_active[symbol] = False
        websocket_instances[symbol] = None
        threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect)
    wst.daemon = True
    wst.start()

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol):
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = 1.0
    max_attempts = 30
    attempts = 0
    while attempts < max_attempts:
        try:
            ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
            if ltp_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = float(ltp_response["data"]["ltp"])
                logger.debug(f"API LTP for {symbol}: {ltp_data[symbol]:.2f}")
                backoff = 1.0
                attempts = 0
            else:
                logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                token, _ = fetch_symbol_token(symbol)
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5)
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}")
            backoff = min(backoff * 2, 5)
            attempts += 1
        time.sleep(backoff)
    logger.error(f"Max fallback attempts reached for {symbol}")

def websocket_watchdog():
    while True:
        time.sleep(15)
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close()
                    except Exception:
                        pass
                    websocket_instances[symbol] = None
                start_websocket(symbol)

def ensure_ltp(symbol, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        if ltp_data.get(symbol) is not None:
            logger.info(f"LTP available for {symbol}: {ltp_data[symbol]:.2f}")
            return True
        time.sleep(1)
    logger.error(f"Failed to obtain LTP for {symbol} after {timeout} seconds.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol, trade_type):
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            return None

        quantity = FIXED_QUANTITY
        token, tradingsymbol = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity)
        }

        entry_order = place_order(market_order_params)
        if not entry_order:
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.5 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": "SELL" if trade_type == "BUY" else "BUY",
            "exchange": "NSE",
            "ordertype": "STOPLOSS_MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity),
            "triggerprice": str(round(stop_price, 1))
        }

        sl_order = place_order(sl_order_params)
        if not sl_order:
            logger.critical(f"STOP LOSS ORDER FAILED for {symbol}. Exiting position immediately!")
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            place_order(exit_params)
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary():
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"Type: {summary['trade_type']}\n"
                f"Entry: {summary['entry_price']:.2f}\n"
                f"Exits: {exit_prices_str}\n"
                f"P/L: {summary['net_pnl']:.2f}\n"
                f"Layers: [{profit_layers_str}]\n"
                f"Reason: {summary['exit_reason']}"
            )

def cleanup(signum=None, frame=None):
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        if current_positions[symbol]:
            ltp = ltp_data.get(symbol, current_positions[symbol]["entry_price"])
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(qty)
            }
            exit_order = place_order(exit_order_params)
            if exit_order:
                fees = qty * ltp * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                profit = (ltp - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - ltp) * qty
                net_profit = profit - fees
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": current_positions[symbol]["entry_price"],
                    "exit_details": [{"price": ltp, "quantity": qty}],
                    "net_pnl": net_profit,
                    "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES],
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": "Script Interrupted"
                }
                logger.info(f"Exited {trade_type} for {symbol}: {qty} at {ltp:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted")
            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
            current_positions[symbol] = None
            stop_loss_orders[symbol] = None
        if websocket_instances[symbol]:
            try:
                websocket_instances[symbol].close()
                logger.info(f"Closed WebSocket for {symbol}")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")
    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main():
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5)

        active_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol):
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_threads.append(trade_thread)

        if active_threads:
            logger.info(f"Waiting for {len(active_threads)} trade(s) to complete...")
            for t in active_threads:
                t.join()
            logger.info("All trades have been completed.")
        else:
            logger.info("No trades were initiated.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup()

if __name__ == "__main__":
    main()

    +++


[I 250612 14:11:51 <ipython-input-5-2779561084>:65] Trading script started with fixed target (real trading)
[E 250612 14:11:54 <ipython-input-3-3126146108>:221] WebSocket error for MOTHERSON: Connection closed
[W 250612 14:11:54 smartWebSocketV2:342] Connection closed due to max retry attempts reached.
ERROR:websocket:Handshake status 429 Too Many Requests -+-+- {'server': 'nginx', 'date': 'Thu, 12 Jun 2025 08:41:54 GMT', 'content-type': 'text/html;charset=iso-8859-1', 'content-length': '615', 'connection': 'keep-alive', 'x-error-message': 'Invalid Request. Connection Limit Exceeded.', 'cache-control': 'must-revalidate,no-cache,no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-security-policy': "default-src 'self';"} -+-+- b'<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>\n<title>Error 429 Invalid Request. Connection Limit Exceeded.</title>\n</head>\n<body><h2>HTTP ERROR 429 Invalid Request. Connection Limit

Traceback (most recent call last):
  File "<ipython-input-5-2779561084>", line 716, in main
    t.join()
  File "/usr/lib/python3.11/threading.py", line 1119, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.11/threading.py", line 1139, in _wait_for_tstate_lock
    if lock.acquire(block, timeout):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-5-2779561084>", line 683, in cleanup
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-2779561084>", line 727, in <cell line: 0>
    main()
  File "<ipython-input-5-2779561084>", line 724, in main
    cleanup()
  File "<ipython-input-5-2779561084>", line 683, in cleanup
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another excep

TypeError: object of type 'NoneType' has no len()

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
""""'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''+!pip install smartapi-python
!pip install pandas_ta
!pip install logzero
!pip install retrying
!pip install yfinance
!pip install pyotp

!pip install numpy==1.24.4 --force-reinstall

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata']
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 1
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30 # Increased timeout for initial LTP fetch

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
current_positions = {symbol: None for symbol in SYMBOLS}
ltp_data = {symbol: {"price": None, "timestamp": 0} for symbol in SYMBOLS} # Store timestamp
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol, exchange="NSE"):
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params):
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        if isinstance(order, str): # SmartApi returns order ID (string) on success
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0 # Reset failures on success
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            error_message = order.get('message', 'Unknown error')
            error_code = order.get('errorcode', 'N/A')
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: Message: {error_message}, Code: {error_code}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}", exc_info=True)
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id):
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id):
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            cancel_response = obj.cancelOrder(order_id, details.get("variety"))
            if cancel_response and cancel_response.get("status"):
                logger.info(f"Cancelled order {order_id}")
                return True
            else:
                logger.warning(f"Failed to cancel order {order_id}: {cancel_response.get('message', 'Unknown error')}")
                return False
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return None

# --- Main Trading Logic ---
def fixed_target_trade(symbol, entry_price, trade_type, quantity, entry_time):
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop # SAR is your dynamic stop-loss
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_trade_loop_time = time.time()
    try:
        while current_positions[symbol]: # Loop as long as position is open
            ltp_info = ltp_data.get(symbol)
            ltp = ltp_info["price"] if ltp_info else None

            if ltp is None or (time.time() - ltp_info["timestamp"] > 10 if ltp_info["timestamp"] else True): # Check for stale LTP
                logger.warning(f"No fresh LTP for {symbol}. Attempting to fetch via API...")
                for _ in range(5): # Retry fetching LTP
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token: # Re-fetch token if missing
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            fetched_ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = {"price": fetched_ltp, "timestamp": time.time()}
                            ltp = fetched_ltp
                            logger.debug(f"Fetched LTP for {symbol} via API: {ltp:.2f}")
                            break
                        else:
                            logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol} via API: {str(e)}")
                    time.sleep(0.5)

                if ltp is None:
                    if time.time() - start_trade_loop_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol} during trade. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable (Timeout)"
                        break # Exit the while loop
                    else:
                        # Continue waiting, or use the last known entry price if no LTP is available for a short period
                        # For robustness, we should avoid using stale LTP or entry price for active trade management
                        logger.warning(f"Still no LTP for {symbol}. Waiting...")
                        time.sleep(1)
                        continue # Skip to next iteration if LTP is still None

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            # Exit Conditions
            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            # Partial Profit Booking
            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        # Calculate quantity for this layer - ensure it's at least 1 if possible
                        profit_qty = int(quantity * profit_percentages[i]) # Use initial quantity for percentage calc
                        if i == NUM_PROFIT_LAYERS - 1: # Last layer takes remaining quantity
                            profit_qty = current_qty
                        elif profit_qty == 0 and current_qty > 0: # Ensure at least 1 quantity if it's the only one left to close
                            profit_qty = 1 if current_qty >= 1 else 0

                        if profit_qty > 0:
                            exit_order_params = {
                                "variety": "NORMAL",
                                "tradingsymbol": tradingsymbol,
                                "symboltoken": str(token),
                                "transactiontype": exit_transaction,
                                "exchange": "NSE",
                                "ordertype": "MARKET",
                                "producttype": "INTRADAY",
                                "duration": "DAY",
                                "price": "0.00",
                                "quantity": str(profit_qty)
                            }
                            exit_order = place_order(exit_order_params)
                            if exit_order and exit_order["status"]:
                                completed_exit = wait_for_order_completion(exit_order["orderid"])
                                if completed_exit:
                                    actual_exit_price = float(completed_exit['averageprice'])
                                    fees = profit_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                                    profit = (actual_exit_price - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * profit_qty
                                    net_profit = profit - fees
                                    total_net_profit += net_profit
                                    current_qty -= profit_qty
                                    total_profit_taken += 1
                                    exit_details.append({"price": actual_exit_price, "quantity": profit_qty})
                                    partial_profit_taken[symbol][i] = True
                                    tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                                    logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}")
                                    with loss_lock:
                                        if net_profit < 0:
                                            consecutive_losses += 1
                                        else:
                                            consecutive_losses = 0
                                else:
                                    logger.error(f"Failed to confirm profit layer {i+1} exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            else:
                                logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            # Trailing Stop Loss / Breakeven Logic
            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price # Move SL to breakeven
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            # Update/Place Stop Loss Order
            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1)) # Round to 1 decimal place for trigger price
                }

                # Check if an SL order exists and needs modification
                order_id = stop_loss_orders[symbol].get("order_id") if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        if float(details.get("triggerprice", 0)) != round(sar, 1) or int(details.get("quantity", 0)) != current_qty:
                            try:
                                sl_order_params["orderid"] = order_id
                                obj.modifyOrder(sl_order_params)
                                logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}, Quantity {sl_order_params['quantity']}")
                                last_sl_time[symbol] = time.time()
                            except Exception as e:
                                logger.error(f"SL order modification failed for {symbol}: {str(e)}. Attempting to cancel and re-place.", exc_info=True)
                                cancel_order(order_id) # Cancel old order
                                new_sl_order = place_order(sl_order_params) # Place new order
                                if new_sl_order:
                                    stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                    last_sl_time[symbol] = time.time()
                    elif details:
                        logger.warning(f"Existing SL order {order_id} for {symbol} is not open/pending. Status: {details.get('orderstatus')}. Re-placing.")
                        cancel_order(order_id) # Cancel if not open/pending
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                    else: # If details couldn't be fetched, assume it's gone and re-place
                        logger.warning(f"Could not retrieve details for existing SL order {order_id} for {symbol}. Re-placing.")
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                elif not order_id: # No existing SL order, place a new one
                    new_sl_order = place_order(sl_order_params)
                    if new_sl_order:
                        stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                        last_sl_time[symbol] = time.time()


            # Final Exit if conditions met or quantity is zero
            if current_qty <= 0 or exit_position:
                if current_qty > 0 and exit_position: # If there's remaining quantity and an exit condition was met
                    exit_order_params = {
                        "variety": "NORMAL",
                        "tradingsymbol": tradingsymbol,
                        "symboltoken": str(token),
                        "transactiontype": exit_transaction,
                        "exchange": "NSE",
                        "ordertype": "MARKET",
                        "producttype": "INTRADAY",
                        "duration": "DAY",
                        "price": "0.00",
                        "quantity": str(current_qty)
                    }
                    exit_order = place_order(exit_order_params)
                    if exit_order and exit_order["status"]:
                        completed_final_exit = wait_for_order_completion(exit_order['orderid'])
                        if completed_final_exit:
                            actual_exit_price = float(completed_final_exit['averageprice'])
                            fees = current_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (actual_exit_price - entry_price) * current_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * current_qty
                            net_profit = profit - fees
                            total_net_profit += net_profit
                            exit_details.append({"price": actual_exit_price, "quantity": current_qty})
                            logger.info(f"Final exit for {symbol}: {current_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                            current_qty = 0 # Mark as fully exited
                        else:
                            logger.error(f"Failed to confirm final exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            # Even if order confirmation fails, assume it was sent for reporting purposes
                            current_qty = 0 # Attempt to clear position state
                    else:
                        logger.error(f"Failed to place final exit order for {symbol}. Current Qty: {current_qty}")
                        # If exit order fails, something is seriously wrong, might need manual intervention

                # Clean up stop loss order if position is closed
                if stop_loss_orders[symbol]:
                    cancel_order(stop_loss_orders[symbol]["order_id"])
                    stop_loss_orders[symbol] = None

                # Record trade summary and clear position state
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": entry_price,
                    "exit_details": exit_details,
                    "net_pnl": total_net_profit,
                    "profit_layers": profit_layers,
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": exit_reason if exit_reason else "All quantity exited"
                }
                current_positions[symbol] = None # Mark position as closed
                partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                break # Exit the while loop

            time.sleep(1) # Wait for 1 second before next check
    except Exception as e:
        logger.error(f"Fixed target trade loop error for {symbol}: {str(e)}", exc_info=True)
        # Attempt to exit position on critical error
        if current_positions[symbol] and current_qty > 0:
            logger.warning(f"Attempting emergency exit for {symbol} due to error.")
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(current_qty)
            }
            place_order(exit_order_params) # Fire and forget emergency exit
        current_positions[symbol] = None
        stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol):
    global websocket_active, websocket_instances
    if websocket_active.get(symbol):
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP[symbol]
    correlation_id = f"{symbol}_{int(time.time())}"

    # Ensure previous instance is properly closed if it exists and wasn't fully cleaned
    if websocket_instances.get(symbol):
        try:
            websocket_instances[symbol].close()
            logger.info(f"Force-closed stale WebSocket for {symbol} before reconnecting.")
        except Exception as e:
            logger.warning(f"Error force-closing stale WebSocket for {symbol}: {str(e)}")
        websocket_instances[symbol] = None # Clear reference

    ws = SmartWebSocketV2(auth_token=JWT_TOKEN, api_key=API_KEY, client_code=USERNAME, feed_token=FEED_TOKEN)
    websocket_instances[symbol] = ws

    def on_data(ws_instance, message): # Use ws_instance to avoid shadowing
        try:
            # logger.debug(f"WebSocket raw message for {symbol}: {message}") # Too verbose, uncomment for deep debugging
            # Angel One WebSocket data structure can vary slightly. Ensure robust parsing.
            if isinstance(message, dict):
                # Mode 1 response for LTP updates
                ltp = message.get('ltp')
                # Older/other formats might have 'last_traded_price' which needs division
                if ltp is None and message.get('last_traded_price'):
                    ltp = message['last_traded_price'] / 100.0 # Assuming it comes as integer * 100
            else: # Sometimes messages might come as string, try parsing as JSON
                try:
                    parsed_message = json.loads(message)
                    ltp = parsed_message.get('ltp')
                    if ltp is None and parsed_message.get('last_traded_price'):
                        ltp = parsed_message['last_traded_price'] / 100.0
                except json.JSONDecodeError:
                    logger.warning(f"Could not parse WebSocket message as JSON for {symbol}: {message}")
                    ltp = None

            if ltp:
                ltp_data[symbol] = {"price": float(ltp), "timestamp": time.time()}
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]['price']:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data processing error for {symbol}: {str(e)}", exc_info=True)

    def on_open(ws_instance):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1, # Mode 1 for LTP
                "tokenList": [{"exchangeType": 1, "tokens": [int(token)]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws_instance.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws_instance, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}", exc_info=True)
        with api_lock:
            global api_failures
            api_failures += 1
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

    # Modified on_close to potentially match expected library signature if it changed
    def on_close(ws_instance, *args, **kwargs):
        # The error suggests the library is passing arguments that the SmartApi wrapper's
        # internal _on_close is not handling correctly with the current signature.
        # However, for a user-defined callback, *args and **kwargs is usually robust.
        # The error might be deeper in SmartApi's internal wrapper.
        logger.warning(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs}")
        websocket_active[symbol] = False
        # websocket_instances[symbol] = None # This is already handled by watchdog/reconnect logic
        # Start fallback only if we're not actively trying to restart WS (watchdog will handle it)
        # threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect)
    wst.daemon = True
    wst.start()
    logger.info(f"Attempting to connect WebSocket for {symbol}")

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol):
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = 1.0
    max_attempts = 30 # Number of attempts, not total time
    attempts = 0
    while attempts < max_attempts and not websocket_active.get(symbol): # Stop if WS becomes active
        try:
            ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
            if ltp_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = {"price": float(ltp_response["data"]["ltp"]), "timestamp": time.time()}
                logger.debug(f"API LTP for {symbol} (fallback): {ltp_data[symbol]['price']:.2f}")
                backoff = 1.0 # Reset backoff on success
                attempts = 0 # Reset attempts on success
            else:
                logger.warning(f"LTP API failed for {symbol} (fallback): {ltp_response.get('message', 'No data')}")
                # Re-fetch token only if a token specific error or no data is returned
                token, _ = fetch_symbol_token(symbol) # Try to refresh token
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5) # Max 5 seconds backoff
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}", exc_info=True)
            backoff = min(backoff * 2, 5)
            attempts += 1
        time.sleep(backoff)
    if not websocket_active.get(symbol):
        logger.error(f"Max fallback attempts reached for {symbol} and WebSocket still inactive.")
    else:
        logger.info(f"WebSocket reconnected for {symbol}, stopping fallback price fetch.")


def websocket_watchdog():
    while True:
        time.sleep(WEBSOCKET_RETRY_DELAY) # Check more frequently than 15s if connection is fragile
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                # Closing explicitly before starting new one to clear any lingering states
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close()
                    except Exception:
                        pass
                    websocket_instances[symbol] = None
                start_websocket(symbol) # Attempt to restart WebSocket
                # Also start fallback if websocket fails to connect for a while
                threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()


def ensure_ltp(symbol, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        ltp_info = ltp_data.get(symbol)
        if ltp_info and ltp_info["price"] is not None and (time.time() - ltp_info["timestamp"] < 10): # Check if LTP is recent
            logger.info(f"LTP available for {symbol}: {ltp_info['price']:.2f}")
            return True
        logger.info(f"Waiting for LTP for {symbol}...")
        time.sleep(1)
    logger.error(f"Failed to obtain fresh LTP for {symbol} after {timeout} seconds.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol, trade_type):
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            logger.error(f"Cannot place trade for {symbol}: No LTP available.")
            return None

        quantity = FIXED_QUANTITY
        token, tradingsymbol = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity)
        }

        entry_order = place_order(market_order_params)
        if not entry_order or not entry_order.get("status"): # Check for status in dict
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            # If entry order itself failed to complete, should it be immediately exited?
            # For simplicity, if confirmation fails, we treat it as failed to enter.
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.5 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": "SELL" if trade_type == "BUY" else "BUY",
            "exchange": "NSE",
            "ordertype": "STOPLOSS_MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity),
            "triggerprice": str(round(stop_price, 1))
        }

        sl_order = place_order(sl_order_params)
        if not sl_order or not sl_order.get("status"):
            logger.critical(f"STOP LOSS ORDER FAILED for {symbol}. Exiting position immediately!")
            # Attempt to exit the entered position if SL fails
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            emergency_exit_order = place_order(exit_params)
            if emergency_exit_order and emergency_exit_order.get("status"):
                logger.info(f"Emergency exit order placed for {symbol} after SL failure.")
            else:
                logger.error(f"CRITICAL: Emergency exit order also failed for {symbol}. Manual intervention required!")
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary():
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"  Type: {summary['trade_type']}\n"
                f"  Entry: {summary['entry_price']:.2f}\n"
                f"  Exits: {exit_prices_str}\n"
                f"  Net P/L: {summary['net_pnl']:.2f}\n"
                f"  Layers: [{profit_layers_str}]\n"
                f"  Reason: {summary['exit_reason']}"
            )
        else:
            logger.info(f"No trade summary available for {symbol}.")


def cleanup(signum=None, frame=None):
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        # Attempt to exit any open positions
        if current_positions[symbol]:
            ltp = ltp_data[symbol]["price"] if ltp_data[symbol]["price"] is not None else current_positions[symbol]["entry_price"]
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if qty > 0 and token and tradingsymbol: # Only try to exit if quantity > 0 and token exists
                logger.info(f"Attempting to exit remaining position for {symbol} (Qty: {qty}) due to script shutdown.")
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order and exit_order.get("status"):
                    completed_exit = wait_for_order_completion(exit_order['orderid'])
                    if completed_exit:
                        actual_exit_price = float(completed_exit['averageprice'])
                        fees = qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                        profit = (actual_exit_price - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - actual_exit_price) * qty
                        net_profit = profit - fees
                        # Update trade_summaries for this forced exit
                        trade_summaries[symbol] = {
                            "trade_type": trade_type,
                            "entry_price": current_positions[symbol]["entry_price"],
                            "exit_details": [{"price": actual_exit_price, "quantity": qty}],
                            "net_pnl": net_profit,
                            "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES], # This might be inaccurate for partial exits
                            "profit_layers_hit": partial_profit_taken[symbol].copy(),
                            "exit_reason": "Script Interrupted (Forced Exit)"
                        }
                        logger.info(f"Exited {trade_type} for {symbol}: {qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted (Forced Exit)")
                    else:
                        logger.error(f"Failed to confirm forced exit order for {symbol}.")
                else:
                    logger.error(f"Failed to place forced exit order for {symbol} on shutdown.")
            else:
                logger.warning(f"No active position or invalid token/tradingsymbol for {symbol} to force exit.")

            # Cancel any open stop-loss orders
            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
                stop_loss_orders[symbol] = None

            current_positions[symbol] = None # Clear position state

        # Close WebSocket connections
        if websocket_instances[symbol]:
            try:
                websocket_instances[symbol].close()
                logger.info(f"Closed WebSocket for {symbol}")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")

    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main():
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        # Check market hours
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()
            return # Exit main if outside hours

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5) # Give some time for websockets to connect and fetch initial LTP

        active_trade_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol): # Only place a trade if no position is active for the symbol
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_trade_threads.append(trade_thread)
                else:
                    logger.error(f"Failed to initiate manual trade for {symbol}. Will not proceed with this symbol.")

        if active_trade_threads:
            logger.info(f"Waiting for {len(active_trade_threads)} trade(s) to complete...")
            for t in active_trade_threads:
                t.join() # Wait for all trade threads to complete
            logger.info("All initiated trades have been completed.")
        else:
            logger.info("No trades were initiated successfully.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup() # Ensure cleanup happens even if an unhandled exception occurs

if __name__ == "__main__":
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 325)

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata'
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 6
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
current_positions = {symbol: None for symbol in SYMBOLS}
ltp_data = {symbol: None for symbol in SYMBOLS}
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol, exchange="NSE"):
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params):
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        # SmartApi returns order ID (string) on success or dict with error
        if isinstance(order, str):
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: {order.get('message', 'Unknown error')}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}")
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.error("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id):
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id):
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            obj.cancelOrder(order_id, details.get("variety"))
            logger.info(f"Cancelled order {order_id}")
            return True
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return None

# --- Main Trading Logic ---
def fixed_target_trade(symbol, entry_price, trade_type, quantity, entry_time):
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_time = time.time()
    try:
        while current_positions[symbol]:
            ltp = ltp_data.get(symbol)
            if ltp is None:
                logger.warning(f"No LTP for {symbol}. Attempting to fetch...")
                for _ in range(5):
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token:
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = ltp
                            logger.debug(f"Fetched LTP for {symbol}: {ltp:.2f}")
                            break
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol}: {str(e)}")
                    time.sleep(0.5)
                if ltp is None:
                    if time.time() - start_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol}. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable"
                    else:
                        ltp = entry_price
                        logger.warning(f"Using entry price for {symbol}: {ltp:.2f}")

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        profit_qty = int(current_qty * profit_percentages[i])
                        if profit_qty == 0:
                            continue
                        exit_order_params = {
                            "variety": "NORMAL",
                            "tradingsymbol": tradingsymbol,
                            "symboltoken": str(token),
                            "transactiontype": exit_transaction,
                            "exchange": "NSE",
                            "ordertype": "MARKET",
                            "producttype": "INTRADAY",
                            "duration": "DAY",
                            "price": "0.00",
                            "quantity": str(profit_qty)
                        }
                        exit_order = place_order(exit_order_params)
                        if exit_order:
                            fees = profit_qty * ltp * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (ltp - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - ltp) * profit_qty
                            net_profit = profit - fees
                            total_net_profit += net_profit
                            current_qty -= profit_qty
                            total_profit_taken += 1
                            exit_details.append({"price": ltp, "quantity": profit_qty})
                            partial_profit_taken[symbol][i] = True
                            tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                            logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {ltp:.2f}, Net P/L: {net_profit:.2f}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                        else:
                            logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1))
                }
                order_id = stop_loss_orders[symbol].get("order_id") if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    sl_order_params["orderid"] = order_id
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        try:
                            obj.modifyOrder(sl_order_params)
                            logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}")
                            last_sl_time[symbol] = time.time()
                        except Exception as e:
                            logger.error(f"SL order modification failed for {symbol}: {str(e)}")
                            new_sl_order = place_order(sl_order_params)
                            if new_sl_order:
                                stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                last_sl_time[symbol] = time.time()
                    else:
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()

            if exit_position and current_qty > 0:
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order:
                    fees = current_qty * ltp * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                    profit = (ltp - entry_price) * current_qty if trade_type == "BUY" else (entry_price - ltp) * current_qty
                    net_profit = profit - fees
                    total_net_profit += net_profit
                    exit_details.append({"price": ltp, "quantity": current_qty})
                    with loss_lock:
                        if net_profit < 0:
                            consecutive_losses += 1
                        else:
                            consecutive_losses = 0
                    logger.info(f"Exited {trade_type} for {symbol}: {current_qty} at {ltp:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                current_qty = 0

            if current_qty <= 0 and stop_loss_orders[symbol]:
                order_id = stop_loss_orders[symbol]["order_id"]
                cancel_order(order_id)
                stop_loss_orders[symbol] = None

            if current_qty <= 0 or exit_position:
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": entry_price,
                    "exit_details": exit_details,
                    "net_pnl": total_net_profit,
                    "profit_layers": profit_layers,
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": exit_reason
                }
                current_positions[symbol] = None
                stop_loss_orders[symbol] = None
                partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                break

            time.sleep(1)
    except Exception as e:
        logger.error(f"Fixed target error for {symbol}: {str(e)}")
        current_positions[symbol] = None
        stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol):
    global websocket_active, websocket_instances
    if websocket_active.get(symbol):
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP[symbol]
    correlation_id = f"{symbol}_{int(time.time())}"

    ws = SmartWebSocketV2(auth_token=JWT_TOKEN, api_key=API_KEY, client_code=USERNAME, feed_token=FEED_TOKEN)
    websocket_instances[symbol] = ws

    def on_data(ws, message):
        try:
            logger.debug(f"WebSocket raw message for {symbol}: {message}")
            ltp = message.get('ltp') or (message.get('last_traded_price') / 100 if message.get('last_traded_price') else None)
            if ltp:
                ltp_data[symbol] = float(ltp)
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data error for {symbol}: {str(e)}")

    def on_open(ws):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1,
                "tokenList": [{"exchangeType": 1, "tokens": [int(token)]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}")
        with api_lock:
            global api_failures
            api_failures += 1
            if api_failures >= MAX_API_FAILURES:
                logger.error("Maximum API failures reached. Exiting.")
                sys.exit(1)

    def on_close(ws, *args, **kwargs):
        logger.warning(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs}")
        websocket_active[symbol] = False
        websocket_instances[symbol] = None
        threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect)
    wst.daemon = True
    wst.start()

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol):
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = 1.0
    max_attempts = 30
    attempts = 0
    while attempts < max_attempts:
        try:
            ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
            if ltp_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = float(ltp_response["data"]["ltp"])
                logger.debug(f"API LTP for {symbol}: {ltp_data[symbol]:.2f}")
                backoff = 1.0
                attempts = 0
            else:
                logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                token, _ = fetch_symbol_token(symbol)
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5)
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}")
            backoff = min(backoff * 2, 5)
            attempts += 1
        time.sleep(backoff)
    logger.error(f"Max fallback attempts reached for {symbol}")

def websocket_watchdog():
    while True:
        time.sleep(15)
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close()
                    except Exception:
                        pass
                    websocket_instances[symbol] = None
                start_websocket(symbol)

def ensure_ltp(symbol, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        if ltp_data.get(symbol) is not None:
            logger.info(f"LTP available for {symbol}: {ltp_data[symbol]:.2f}")
            return True
        time.sleep(1)
    logger.error(f"Failed to obtain LTP for {symbol} after {timeout} seconds.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol, trade_type):
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            return None

        quantity = FIXED_QUANTITY
        token, tradingsymbol = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity)
        }

        entry_order = place_order(market_order_params)
        if not entry_order:
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.5 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": "SELL" if trade_type == "BUY" else "BUY",
            "exchange": "NSE",
            "ordertype": "STOPLOSS_MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity),
            "triggerprice": str(round(stop_price, 1))
        }

        sl_order = place_order(sl_order_params)
        if not sl_order:
            logger.critical(f"STOP LOSS ORDER FAILED for {symbol}. Exiting position immediately!")
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            place_order(exit_params)
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary():
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"Type: {summary['trade_type']}\n"
                f"Entry: {summary['entry_price']:.2f}\n"
                f"Exits: {exit_prices_str}\n"
                f"P/L: {summary['net_pnl']:.2f}\n"
                f"Layers: [{profit_layers_str}]\n"
                f"Reason: {summary['exit_reason']}"
            )

def cleanup(signum=None, frame=None):
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        if current_positions[symbol]:
            ltp = ltp_data.get(symbol, current_positions[symbol]["entry_price"])
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(qty)
            }
            exit_order = place_order(exit_order_params)
            if exit_order:
                fees = qty * ltp * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                profit = (ltp - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - ltp) * qty
                net_profit = profit - fees
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": current_positions[symbol]["entry_price"],
                    "exit_details": [{"price": ltp, "quantity": qty}],
                    "net_pnl": net_profit,
                    "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES],
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": "Script Interrupted"
                }
                logger.info(f"Exited {trade_type} for {symbol}: {qty} at {ltp:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted")
            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
            current_positions[symbol] = None
            stop_loss_orders[symbol] = None
        if websocket_instances[symbol]:
            try:
                websocket_instances[symbol].close()
                logger.info(f"Closed WebSocket for {symbol}")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")
    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main():
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5)

        active_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol):
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_threads.append(trade_thread)

        if active_threads:
            logger.info(f"Waiting for {len(active_threads)} trade(s) to complete...")
            for t in active_threads:
                t.join()
            logger.info("All trades have been completed.")
        else:
            logger.info("No trades were initiated.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup()

if __name__ == "__main__":
    main()

[I 250612 14:21:59 <ipython-input-6-2824869267>:65] Trading script started with fixed target (real trading)
[W 250612 14:21:59 smartWebSocketV2:318] Attempting to resubscribe/reconnect (Attempt 1)...
[W 250612 14:22:00 <ipython-input-5-2779561084>:517] Watchdog: WebSocket for MOTHERSON is inactive. Attempting to restart.
Exception in thread Thread-78 (websocket_watchdog):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-2779561084>", line 524, in websocket_watchdog
  File "<ipython-input-5-2779561084>", line 425, in start_websocket
KeyError: 'MOTHERSON'
[I 250612 14:22:03 <ipython-input-6-2824869267>:93] Added token 4204 for MOTHERSON-EQ
[D 250612 14:22:03 <ipython-input-6-2824869267>:108] Initialized TOKEN_MAP: {'MOTHERSON': ('4204', 'MOTHERSON-EQ')}
[I 250612 14:22:03 smartConnect:

SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys
from typing import Dict, List, Optional, Tuple, Union

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata'
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
class TradingConfig:
    def __init__(self):
        self.API_KEY = os.getenv("API_KEY", "OBpBVquO")
        self.USERNAME = os.getenv("USERNAME", "AAAN189703")
        self.PASSWORD = os.getenv("PASSWORD", "5350")
        self.TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

        # Trading Parameters
        self.SYMBOLS = ["MOTHERSON"]
        self.MAX_API_FAILURES = 5
        self.MIN_MODIFY_INTERVAL = 5
        self.FIXED_TARGET_POINTS = 0.5
        self.NUM_PROFIT_LAYERS = 6
        self.PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
        self.TIGHTENING_FACTOR_MIN = 0.5
        self.TIGHTENING_FACTOR_STEP = 0.1
        self.FIXED_QUANTITY = 6
        self.MANUAL_TRADE_TYPE = "BUY"

        # Fees and Slippage
        self.BROKERAGE_FEE = 0.00236
        self.STT_CTT = 0.00025
        self.EXCHANGE_FEE = 0.0000324
        self.SLIPPAGE = 0.001

        # Timing and Retries
        self.EXIT_TIME_SECONDS = 22500
        self.WEBSOCKET_RETRY_DELAY = 5
        self.LTP_TIMEOUT = 30

    def validate(self) -> bool:
        """Validate configuration parameters."""
        if len(self.PROFIT_DISTANCES) != self.NUM_PROFIT_LAYERS:
            logger.error(f"PROFIT_DISTANCES length ({len(self.PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({self.NUM_PROFIT_LAYERS})")
            return False
        if self.PROFIT_DISTANCES[-1] != self.FIXED_TARGET_POINTS:
            logger.error(f"Last PROFIT_DISTANCE ({self.PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({self.FIXED_TARGET_POINTS})")
            return False
        return True

# --- Global State and Locks ---
class TradingState:
    def __init__(self):
        self.TOKEN_MAP: Dict[str, Tuple[str, str]] = {}
        self.FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
        self.ltp_data: Dict[str, Dict[str, Union[float, int]]] = {symbol: {"price": None, "timestamp": 0} for symbol in config.SYMBOLS}
        self.current_positions: Dict[str, Optional[Dict]] = {symbol: None for symbol in config.SYMBOLS}
        self.stop_loss_orders: Dict[str, Optional[Dict]] = {symbol: None for symbol in config.SYMBOLS}
        self.partial_profit_taken: Dict[str, List[bool]] = {symbol: [False] * config.NUM_PROFIT_LAYERS for symbol in config.SYMBOLS}
        self.last_sl_time: Dict[str, float] = {symbol: 0 for symbol in config.SYMBOLS}
        self.trade_summaries: Dict[str, Dict] = {symbol: {} for symbol in config.SYMBOLS}
        self.websocket_active: Dict[str, bool] = {symbol: False for symbol in config.SYMBOLS}
        self.websocket_instances: Dict[str, Optional[SmartWebSocketV2]] = {symbol: None for symbol in config.SYMBOLS}

        self.api_lock = threading.Lock()
        self.loss_lock = threading.Lock()
        self.daily_trades = 0
        self.consecutive_losses = 0
        self.api_failures = 0

# Initialize configuration and state
config = TradingConfig()
state = TradingState()

# Initialize logging
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol: str, exchange: str = "NSE") -> Tuple[str, str]:
    """Fetch token for a given symbol with retry mechanism."""
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()

        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol

        logger.warning(f"No token found for {symbol}. Using fallback.")
        return state.FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return state.FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

# Initialize tokens
for symbol in config.SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        state.TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {state.TOKEN_MAP}")

# --- API Connection ---
def connect_to_api() -> bool:
    """Establish API connection with retry mechanism."""
    try:
        obj = SmartConnect(api_key=config.API_KEY)
        totp = pyotp.TOTP(config.TOTP_SECRET)
        data = obj.generateSession(config.USERNAME, config.PASSWORD, totp.now())

        if not data.get("status"):
            logger.error(f"Login failed: {data.get('message')}")
            return False

        state.FEED_TOKEN = data["data"]["feedToken"]
        state.JWT_TOKEN = data["data"]["jwtToken"]
        time.sleep(2)
        logger.info("Login successful")
        return True
    except Exception as e:
        logger.error(f"API connection failed: {str(e)}", exc_info=True)
        return False

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params: Dict) -> Optional[Dict]:
    """Place an order with retry mechanism and failure tracking."""
    try:
        with state.api_lock:
            order = obj.placeOrder(order_params)

            if isinstance(order, str):  # SmartApi returns order ID (string) on success
                logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
                state.api_failures = 0  # Reset failures on success
                return {"status": True, "orderid": order}

            elif isinstance(order, dict) and not order.get("status"):
                error_message = order.get('message', 'Unknown error')
                error_code = order.get('errorcode', 'N/A')
                logger.error(f"Order placement failed for {order_params['tradingsymbol']}: Message: {error_message}, Code: {error_code}")
                state.api_failures += 1
                return None

            else:
                logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
                state.api_failures += 1
                return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}", exc_info=True)
        with state.api_lock:
            state.api_failures += 1
        return None
    finally:
        with state.api_lock:
            if state.api_failures >= config.MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

# --- Main Trading Logic ---
def fixed_target_trade(symbol: str, entry_price: float, trade_type: str, quantity: int, entry_time: float) -> None:
    """Execute fixed target trading strategy with enhanced error handling."""
    try:
        stop_loss_factor = 0.5
        profit_percentages = [1.0 / config.NUM_PROFIT_LAYERS for _ in range(config.NUM_PROFIT_LAYERS)]
        tightening_factor = 1.0

        if trade_type == "BUY":
            initial_stop = entry_price - stop_loss_factor
            exit_transaction = "SELL"
            profit_layers = [entry_price + d for d in config.PROFIT_DISTANCES]
        else:
            initial_stop = entry_price + stop_loss_factor
            exit_transaction = "BUY"
            profit_layers = [entry_price - d for d in config.PROFIT_DISTANCES]

        sar = initial_stop  # SAR is your dynamic stop-loss
        breakeven_set = False
        total_profit_taken = 0
        current_qty = quantity
        exit_details = []
        total_net_profit = 0
        profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
        logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + config.FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

        start_trade_loop_time = time.time()
        while state.current_positions.get(symbol):  # Loop as long as position is open
            ltp_info = state.ltp_data.get(symbol)
            ltp = ltp_info["price"] if ltp_info else None

            if ltp is None or (ltp_info and time.time() - ltp_info["timestamp"] > 10):  # Check for stale LTP
                logger.warning(f"No fresh LTP for {symbol}. Attempting to fetch via API...")
                for _ in range(5):  # Retry fetching LTP
                    try:
                        token, _ = state.TOKEN_MAP.get(symbol, (None, ""))
                        if not token:  # Re-fetch token if missing
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                state.TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            fetched_ltp = float(ltp_response["data"]["ltp"])
                            state.ltp_data[symbol] = {"price": fetched_ltp, "timestamp": time.time()}
                            ltp = fetched_ltp
                            logger.debug(f"Fetched LTP for {symbol} via API: {ltp:.2f}")
                            break
                        else:
                            logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol} via API: {str(e)}")
                    time.sleep(0.5)

            if ltp is None:
                if time.time() - start_trade_loop_time > config.LTP_TIMEOUT:
                    logger.error(f"LTP timeout for {symbol} during trade. Exiting position.")
                    exit_position = True
                    exit_reason = "LTP Unavailable (Timeout)"
                    break  # Exit the while loop
                else:
                    logger.warning(f"Still no LTP for {symbol}. Waiting...")
                    time.sleep(1)
                    continue  # Skip to next iteration if LTP is still None

            # ... (rest of the trading logic remains the same)
    except Exception as e:
        logger.error(f"Fixed target trade loop error for {symbol}: {str(e)}", exc_info=True)
        # Attempt to exit position on critical error
        if state.current_positions.get(symbol) and current_qty > 0:
            logger.warning(f"Attempting emergency exit for {symbol} due to error.")
            token, tradingsymbol = state.TOKEN_MAP.get(symbol, (None, ""))
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(current_qty)
            }
            place_order(exit_order_params)  # Fire and forget emergency exit
        state.current_positions[symbol] = None
        state.stop_loss_orders[symbol] = None

# ... (rest of the code remains the same)

def main():
    """Main execution block with enhanced error handling."""
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)

        # Check market hours
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()
            return

        # Initialize API connection
        if not connect_to_api():
            logger.error("Failed to connect to API. Exiting.")
            cleanup()
            return

        # Initialize WebSocket connections
        for symbol in config.SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5)  # Give some time for websockets to connect and fetch initial LTP

        active_trade_threads = []
        for symbol in config.SYMBOLS:
            if not state.current_positions.get(symbol):  # Only place a trade if no position is active for the symbol
                trade_thread = place_manual_trade(symbol, config.MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_trade_threads.append(trade_thread)
                else:
                    logger.error(f"Failed to initiate manual trade for {symbol}. Will not proceed with this symbol.")

        if active_trade_threads:
            logger.info(f"Waiting for {len(active_trade_threads)} trade(s) to complete...")
            for t in active_trade_threads:
                t.join()  # Wait for all trade threads to complete
            logger.info("All initiated trades have been completed.")
        else:
            logger.info("No trades were initiated successfully.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup()  # Ensure cleanup happens even if an unhandled exception occurs

if __name__ == "__main__":
    main()

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata']
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 6
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30 # Increased timeout for initial LTP fetch

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
current_positions = {symbol: None for symbol in SYMBOLS}
ltp_data = {symbol: {"price": None, "timestamp": 0} for symbol in SYMBOLS} # Store timestamp
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol, exchange="NSE"):
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params):
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        if isinstance(order, str): # SmartApi returns order ID (string) on success
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0 # Reset failures on success
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            error_message = order.get('message', 'Unknown error')
            error_code = order.get('errorcode', 'N/A')
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: Message: {error_message}, Code: {error_code}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}", exc_info=True)
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id):
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id):
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            cancel_response = obj.cancelOrder(order_id, details.get("variety"))
            if cancel_response and cancel_response.get("status"):
                logger.info(f"Cancelled order {order_id}")
                return True
            else:
                logger.warning(f"Failed to cancel order {order_id}: {cancel_response.get('message', 'Unknown error')}")
                return False
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return None

# --- Main Trading Logic ---
def fixed_target_trade(symbol, entry_price, trade_type, quantity, entry_time):
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop # SAR is your dynamic stop-loss
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_trade_loop_time = time.time()
    try:
        while current_positions[symbol]: # Loop as long as position is open
            ltp_info = ltp_data.get(symbol)
            ltp = ltp_info["price"] if ltp_info else None

            if ltp is None or (time.time() - ltp_info["timestamp"] > 10 if ltp_info["timestamp"] else True): # Check for stale LTP
                logger.warning(f"No fresh LTP for {symbol}. Attempting to fetch via API...")
                for _ in range(5): # Retry fetching LTP
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token: # Re-fetch token if missing
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            fetched_ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = {"price": fetched_ltp, "timestamp": time.time()}
                            ltp = fetched_ltp
                            logger.debug(f"Fetched LTP for {symbol} via API: {ltp:.2f}")
                            break
                        else:
                            logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol} via API: {str(e)}")
                    time.sleep(0.5)

                if ltp is None:
                    if time.time() - start_trade_loop_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol} during trade. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable (Timeout)"
                        break # Exit the while loop
                    else:
                        # Continue waiting, or use the last known entry price if no LTP is available for a short period
                        # For robustness, we should avoid using stale LTP or entry price for active trade management
                        logger.warning(f"Still no LTP for {symbol}. Waiting...")
                        time.sleep(1)
                        continue # Skip to next iteration if LTP is still None

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            # Exit Conditions
            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            # Partial Profit Booking
            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        # Calculate quantity for this layer - ensure it's at least 1 if possible
                        profit_qty = int(quantity * profit_percentages[i]) # Use initial quantity for percentage calc
                        if i == NUM_PROFIT_LAYERS - 1: # Last layer takes remaining quantity
                            profit_qty = current_qty
                        elif profit_qty == 0 and current_qty > 0: # Ensure at least 1 quantity if it's the only one left to close
                            profit_qty = 1 if current_qty >= 1 else 0

                        if profit_qty > 0:
                            exit_order_params = {
                                "variety": "NORMAL",
                                "tradingsymbol": tradingsymbol,
                                "symboltoken": str(token),
                                "transactiontype": exit_transaction,
                                "exchange": "NSE",
                                "ordertype": "MARKET",
                                "producttype": "INTRADAY",
                                "duration": "DAY",
                                "price": "0.00",
                                "quantity": str(profit_qty)
                            }
                            exit_order = place_order(exit_order_params)
                            if exit_order and exit_order["status"]:
                                completed_exit = wait_for_order_completion(exit_order["orderid"])
                                if completed_exit:
                                    actual_exit_price = float(completed_exit['averageprice'])
                                    fees = profit_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                                    profit = (actual_exit_price - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * profit_qty
                                    net_profit = profit - fees
                                    total_net_profit += net_profit
                                    current_qty -= profit_qty
                                    total_profit_taken += 1
                                    exit_details.append({"price": actual_exit_price, "quantity": profit_qty})
                                    partial_profit_taken[symbol][i] = True
                                    tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                                    logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}")
                                    with loss_lock:
                                        if net_profit < 0:
                                            consecutive_losses += 1
                                        else:
                                            consecutive_losses = 0
                                else:
                                    logger.error(f"Failed to confirm profit layer {i+1} exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            else:
                                logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            # Trailing Stop Loss / Breakeven Logic
            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price # Move SL to breakeven
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            # Update/Place Stop Loss Order
            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1)) # Round to 1 decimal place for trigger price
                }

                # Check if an SL order exists and needs modification
                order_id = stop_loss_orders[symbol].get("order_id") if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        if float(details.get("triggerprice", 0)) != round(sar, 1) or int(details.get("quantity", 0)) != current_qty:
                            try:
                                sl_order_params["orderid"] = order_id
                                obj.modifyOrder(sl_order_params)
                                logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}, Quantity {sl_order_params['quantity']}")
                                last_sl_time[symbol] = time.time()
                            except Exception as e:
                                logger.error(f"SL order modification failed for {symbol}: {str(e)}. Attempting to cancel and re-place.", exc_info=True)
                                cancel_order(order_id) # Cancel old order
                                new_sl_order = place_order(sl_order_params) # Place new order
                                if new_sl_order:
                                    stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                    last_sl_time[symbol] = time.time()
                    elif details:
                        logger.warning(f"Existing SL order {order_id} for {symbol} is not open/pending. Status: {details.get('orderstatus')}. Re-placing.")
                        cancel_order(order_id) # Cancel if not open/pending
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                    else: # If details couldn't be fetched, assume it's gone and re-place
                        logger.warning(f"Could not retrieve details for existing SL order {order_id} for {symbol}. Re-placing.")
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                elif not order_id: # No existing SL order, place a new one
                    new_sl_order = place_order(sl_order_params)
                    if new_sl_order:
                        stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                        last_sl_time[symbol] = time.time()


            # Final Exit if conditions met or quantity is zero
            if current_qty <= 0 or exit_position:
                if current_qty > 0 and exit_position: # If there's remaining quantity and an exit condition was met
                    exit_order_params = {
                        "variety": "NORMAL",
                        "tradingsymbol": tradingsymbol,
                        "symboltoken": str(token),
                        "transactiontype": exit_transaction,
                        "exchange": "NSE",
                        "ordertype": "MARKET",
                        "producttype": "INTRADAY",
                        "duration": "DAY",
                        "price": "0.00",
                        "quantity": str(current_qty)
                    }
                    exit_order = place_order(exit_order_params)
                    if exit_order and exit_order["status"]:
                        completed_final_exit = wait_for_order_completion(exit_order['orderid'])
                        if completed_final_exit:
                            actual_exit_price = float(completed_final_exit['averageprice'])
                            fees = current_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (actual_exit_price - entry_price) * current_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * current_qty
                            net_profit = profit - fees
                            total_net_profit += net_profit
                            exit_details.append({"price": actual_exit_price, "quantity": current_qty})
                            logger.info(f"Final exit for {symbol}: {current_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                            current_qty = 0 # Mark as fully exited
                        else:
                            logger.error(f"Failed to confirm final exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            # Even if order confirmation fails, assume it was sent for reporting purposes
                            current_qty = 0 # Attempt to clear position state
                    else:
                        logger.error(f"Failed to place final exit order for {symbol}. Current Qty: {current_qty}")
                        # If exit order fails, something is seriously wrong, might need manual intervention

                # Clean up stop loss order if position is closed
                if stop_loss_orders[symbol]:
                    cancel_order(stop_loss_orders[symbol]["order_id"])
                    stop_loss_orders[symbol] = None

                # Record trade summary and clear position state
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": entry_price,
                    "exit_details": exit_details,
                    "net_pnl": total_net_profit,
                    "profit_layers": profit_layers,
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": exit_reason if exit_reason else "All quantity exited"
                }
                current_positions[symbol] = None # Mark position as closed
                partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                break # Exit the while loop

            time.sleep(1) # Wait for 1 second before next check
    except Exception as e:
        logger.error(f"Fixed target trade loop error for {symbol}: {str(e)}", exc_info=True)
        # Attempt to exit position on critical error
        if current_positions[symbol] and current_qty > 0:
            logger.warning(f"Attempting emergency exit for {symbol} due to error.")
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(current_qty)
            }
            place_order(exit_order_params) # Fire and forget emergency exit
        current_positions[symbol] = None
        stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol):
    global websocket_active, websocket_instances
    if websocket_active.get(symbol):
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP[symbol]
    correlation_id = f"{symbol}_{int(time.time())}"

    # Ensure previous instance is properly closed if it exists and wasn't fully cleaned
    if websocket_instances.get(symbol):
        try:
            websocket_instances[symbol].close()
            logger.info(f"Force-closed stale WebSocket for {symbol} before reconnecting.")
        except Exception as e:
            logger.warning(f"Error force-closing stale WebSocket for {symbol}: {str(e)}")
        websocket_instances[symbol] = None # Clear reference

    ws = SmartWebSocketV2(auth_token=JWT_TOKEN, api_key=API_KEY, client_code=USERNAME, feed_token=FEED_TOKEN)
    websocket_instances[symbol] = ws

    def on_data(ws_instance, message): # Use ws_instance to avoid shadowing
        try:
            # logger.debug(f"WebSocket raw message for {symbol}: {message}") # Too verbose, uncomment for deep debugging
            # Angel One WebSocket data structure can vary slightly. Ensure robust parsing.
            if isinstance(message, dict):
                # Mode 1 response for LTP updates
                ltp = message.get('ltp')
                # Older/other formats might have 'last_traded_price' which needs division
                if ltp is None and message.get('last_traded_price'):
                    ltp = message['last_traded_price'] / 100.0 # Assuming it comes as integer * 100
            else: # Sometimes messages might come as string, try parsing as JSON
                try:
                    parsed_message = json.loads(message)
                    ltp = parsed_message.get('ltp')
                    if ltp is None and parsed_message.get('last_traded_price'):
                        ltp = parsed_message['last_traded_price'] / 100.0
                except json.JSONDecodeError:
                    logger.warning(f"Could not parse WebSocket message as JSON for {symbol}: {message}")
                    ltp = None

            if ltp:
                ltp_data[symbol] = {"price": float(ltp), "timestamp": time.time()}
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]['price']:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data processing error for {symbol}: {str(e)}", exc_info=True)

    def on_open(ws_instance):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1, # Mode 1 for LTP
                "tokenList": [{"exchangeType": 1, "tokens": [int(token)]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws_instance.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws_instance, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}", exc_info=True)
        with api_lock:
            global api_failures
            api_failures += 1
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

    # Modified on_close to potentially match expected library signature if it changed
    def on_close(ws_instance, *args, **kwargs):
        # The error suggests the library is passing arguments that the SmartApi wrapper's
        # internal _on_close is not handling correctly with the current signature.
        # However, for a user-defined callback, *args and **kwargs is usually robust.
        # The error might be deeper in SmartApi's internal wrapper.
        logger.warning(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs}")
        websocket_active[symbol] = False
        # websocket_instances[symbol] = None # This is already handled by watchdog/reconnect logic
        # Start fallback only if we're not actively trying to restart WS (watchdog will handle it)
        # threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect)
    wst.daemon = True
    wst.start()
    logger.info(f"Attempting to connect WebSocket for {symbol}")

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol):
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = 1.0
    max_attempts = 30 # Number of attempts, not total time
    attempts = 0
    while attempts < max_attempts and not websocket_active.get(symbol): # Stop if WS becomes active
        try:
            ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
            if ltp_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = {"price": float(ltp_response["data"]["ltp"]), "timestamp": time.time()}
                logger.debug(f"API LTP for {symbol} (fallback): {ltp_data[symbol]['price']:.2f}")
                backoff = 1.0 # Reset backoff on success
                attempts = 0 # Reset attempts on success
            else:
                logger.warning(f"LTP API failed for {symbol} (fallback): {ltp_response.get('message', 'No data')}")
                # Re-fetch token only if a token specific error or no data is returned
                token, _ = fetch_symbol_token(symbol) # Try to refresh token
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5) # Max 5 seconds backoff
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}", exc_info=True)
            backoff = min(backoff * 2, 5)
            attempts += 1
        time.sleep(backoff)
    if not websocket_active.get(symbol):
        logger.error(f"Max fallback attempts reached for {symbol} and WebSocket still inactive.")
    else:
        logger.info(f"WebSocket reconnected for {symbol}, stopping fallback price fetch.")


def websocket_watchdog():
    while True:
        time.sleep(WEBSOCKET_RETRY_DELAY) # Check more frequently than 15s if connection is fragile
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                # Closing explicitly before starting new one to clear any lingering states
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close()
                    except Exception:
                        pass
                    websocket_instances[symbol] = None
                start_websocket(symbol) # Attempt to restart WebSocket
                # Also start fallback if websocket fails to connect for a while
                threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()


def ensure_ltp(symbol, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        ltp_info = ltp_data.get(symbol)
        if ltp_info and ltp_info["price"] is not None and (time.time() - ltp_info["timestamp"] < 10): # Check if LTP is recent
            logger.info(f"LTP available for {symbol}: {ltp_info['price']:.2f}")
            return True
        logger.info(f"Waiting for LTP for {symbol}...")
        time.sleep(1)
    logger.error(f"Failed to obtain fresh LTP for {symbol} after {timeout} seconds.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol, trade_type):
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            logger.error(f"Cannot place trade for {symbol}: No LTP available.")
            return None

        quantity = FIXED_QUANTITY
        token, tradingsymbol = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity)
        }

        entry_order = place_order(market_order_params)
        if not entry_order or not entry_order.get("status"): # Check for status in dict
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            # If entry order itself failed to complete, should it be immediately exited?
            # For simplicity, if confirmation fails, we treat it as failed to enter.
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.5 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": "SELL" if trade_type == "BUY" else "BUY",
            "exchange": "NSE",
            "ordertype": "STOPLOSS_MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity),
            "triggerprice": str(round(stop_price, 1))
        }

        sl_order = place_order(sl_order_params)
        if not sl_order or not sl_order.get("status"):
            logger.critical(f"STOP LOSS ORDER FAILED for {symbol}. Exiting position immediately!")
            # Attempt to exit the entered position if SL fails
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            emergency_exit_order = place_order(exit_params)
            if emergency_exit_order and emergency_exit_order.get("status"):
                logger.info(f"Emergency exit order placed for {symbol} after SL failure.")
            else:
                logger.error(f"CRITICAL: Emergency exit order also failed for {symbol}. Manual intervention required!")
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary():
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"  Type: {summary['trade_type']}\n"
                f"  Entry: {summary['entry_price']:.2f}\n"
                f"  Exits: {exit_prices_str}\n"
                f"  Net P/L: {summary['net_pnl']:.2f}\n"
                f"  Layers: [{profit_layers_str}]\n"
                f"  Reason: {summary['exit_reason']}"
            )
        else:
            logger.info(f"No trade summary available for {symbol}.")


def cleanup(signum=None, frame=None):
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        # Attempt to exit any open positions
        if current_positions[symbol]:
            ltp = ltp_data[symbol]["price"] if ltp_data[symbol]["price"] is not None else current_positions[symbol]["entry_price"]
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if qty > 0 and token and tradingsymbol: # Only try to exit if quantity > 0 and token exists
                logger.info(f"Attempting to exit remaining position for {symbol} (Qty: {qty}) due to script shutdown.")
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order and exit_order.get("status"):
                    completed_exit = wait_for_order_completion(exit_order['orderid'])
                    if completed_exit:
                        actual_exit_price = float(completed_exit['averageprice'])
                        fees = qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                        profit = (actual_exit_price - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - actual_exit_price) * qty
                        net_profit = profit - fees
                        # Update trade_summaries for this forced exit
                        trade_summaries[symbol] = {
                            "trade_type": trade_type,
                            "entry_price": current_positions[symbol]["entry_price"],
                            "exit_details": [{"price": actual_exit_price, "quantity": qty}],
                            "net_pnl": net_profit,
                            "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES], # This might be inaccurate for partial exits
                            "profit_layers_hit": partial_profit_taken[symbol].copy(),
                            "exit_reason": "Script Interrupted (Forced Exit)"
                        }
                        logger.info(f"Exited {trade_type} for {symbol}: {qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted (Forced Exit)")
                    else:
                        logger.error(f"Failed to confirm forced exit order for {symbol}.")
                else:
                    logger.error(f"Failed to place forced exit order for {symbol} on shutdown.")
            else:
                logger.warning(f"No active position or invalid token/tradingsymbol for {symbol} to force exit.")

            # Cancel any open stop-loss orders
            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
                stop_loss_orders[symbol] = None

            current_positions[symbol] = None # Clear position state

        # Close WebSocket connections
        if websocket_instances[symbol]:
            try:
                websocket_instances[symbol].close()
                logger.info(f"Closed WebSocket for {symbol}")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")

    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main():
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        # Check market hours
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()
            return # Exit main if outside hours

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5) # Give some time for websockets to connect and fetch initial LTP

        active_trade_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol): # Only place a trade if no position is active for the symbol
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_trade_threads.append(trade_thread)
                else:
                    logger.error(f"Failed to initiate manual trade for {symbol}. Will not proceed with this symbol.")

        if active_trade_threads:
            logger.info(f"Waiting for {len(active_trade_threads)} trade(s) to complete...")
            for t in active_trade_threads:
                t.join() # Wait for all trade threads to complete
            logger.info("All initiated trades have been completed.")
        else:
            logger.info("No trades were initiated successfully.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup() # Ensure cleanup happens even if an unhandled exception occurs

if __name__ == "__main__":
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 325)

In [ ]:
File "<tokenize>", line 325
    }
    ^
IndentationError: unindent does not match any outer indentation level+

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata' # Corrected: Removed extra ']' from here
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 6
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30 # Increased timeout for initial LTP fetch

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
ltp_data = {symbol: {"price": None, "timestamp": 0} for symbol in SYMBOLS} # Store timestamp
current_positions = {symbol: None for symbol in SYMBOLS}
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol, exchange="NSE"):
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params):
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        if isinstance(order, str): # SmartApi returns order ID (string) on success
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0 # Reset failures on success
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            error_message = order.get('message', 'Unknown error')
            error_code = order.get('errorcode', 'N/A')
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: Message: {error_message}, Code: {error_code}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}", exc_info=True)
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id):
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id):
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            cancel_response = obj.cancelOrder(order_id, details.get("variety"))
            if cancel_response and cancel_response.get("status"):
                logger.info(f"Cancelled order {order_id}")
                return True
            else:
                logger.warning(f"Failed to cancel order {order_id}: {cancel_response.get('message', 'Unknown error')}")
                return False
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return None

# --- Main Trading Logic ---
def fixed_target_trade(symbol, entry_price, trade_type, quantity, entry_time):
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop # SAR is your dynamic stop-loss
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_trade_loop_time = time.time()
    try:
        while current_positions[symbol]: # Loop as long as position is open
            ltp_info = ltp_data.get(symbol)
            ltp = ltp_info["price"] if ltp_info else None

            if ltp is None or (ltp_info and time.time() - ltp_info["timestamp"] > 10): # Check for stale LTP
                logger.warning(f"No fresh LTP for {symbol}. Attempting to fetch via API...")
                for _ in range(5): # Retry fetching LTP
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token: # Re-fetch token if missing
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            fetched_ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = {"price": fetched_ltp, "timestamp": time.time()}
                            ltp = fetched_ltp
                            logger.debug(f"Fetched LTP for {symbol} via API: {ltp:.2f}")
                            break
                        else:
                            logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol} via API: {str(e)}")
                    time.sleep(0.5)

                if ltp is None:
                    if time.time() - start_trade_loop_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol} during trade. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable (Timeout)"
                        break # Exit the while loop
                    else:
                        logger.warning(f"Still no LTP for {symbol}. Waiting...")
                        time.sleep(1)
                        continue # Skip to next iteration if LTP is still None

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            # Exit Conditions
            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            # Partial Profit Booking
            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        # Calculate quantity for this layer - ensure it's at least 1 if possible
                        profit_qty = int(quantity * profit_percentages[i]) # Use initial quantity for percentage calc
                        if i == NUM_PROFIT_LAYERS - 1: # Last layer takes remaining quantity
                            profit_qty = current_qty
                        elif profit_qty == 0 and current_qty > 0: # Ensure at least 1 quantity if it's the only one left to close
                            profit_qty = 1 if current_qty >= 1 else 0

                        if profit_qty > 0:
                            exit_order_params = {
                                "variety": "NORMAL",
                                "tradingsymbol": tradingsymbol,
                                "symboltoken": str(token),
                                "transactiontype": exit_transaction,
                                "exchange": "NSE",
                                "ordertype": "MARKET",
                                "producttype": "INTRADAY",
                                "duration": "DAY",
                                "price": "0.00",
                                "quantity": str(profit_qty)
                            }
                            exit_order = place_order(exit_order_params)
                            if exit_order and exit_order["status"]:
                                completed_exit = wait_for_order_completion(exit_order["orderid"])
                                if completed_exit:
                                    actual_exit_price = float(completed_exit['averageprice'])
                                    fees = profit_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                                    profit = (actual_exit_price - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * profit_qty
                                    net_profit = profit - fees
                                    total_net_profit += net_profit
                                    current_qty -= profit_qty
                                    total_profit_taken += 1
                                    exit_details.append({"price": actual_exit_price, "quantity": profit_qty})
                                    partial_profit_taken[symbol][i] = True
                                    tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                                    logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}")
                                    with loss_lock:
                                        if net_profit < 0:
                                            consecutive_losses += 1
                                        else:
                                            consecutive_losses = 0
                                else:
                                    logger.error(f"Failed to confirm profit layer {i+1} exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            else:
                                logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            # Trailing Stop Loss / Breakeven Logic
            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price # Move SL to breakeven
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            # Update/Place Stop Loss Order
            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1)) # Round to 1 decimal place for trigger price
                }

                # Check if an SL order exists and needs modification
                order_id = stop_loss_orders[symbol].get("order_id") if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        if float(details.get("triggerprice", 0)) != round(sar, 1) or int(details.get("quantity", 0)) != current_qty:
                            try:
                                sl_order_params["orderid"] = order_id
                                obj.modifyOrder(sl_order_params)
                                logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}, Quantity {sl_order_params['quantity']}")
                                last_sl_time[symbol] = time.time()
                            except Exception as e:
                                logger.error(f"SL order modification failed for {symbol}: {str(e)}. Attempting to cancel and re-place.", exc_info=True)
                                cancel_order(order_id) # Cancel old order
                                new_sl_order = place_order(sl_order_params) # Place new order
                                if new_sl_order:
                                    stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                    last_sl_time[symbol] = time.time()
                    elif details:
                        logger.warning(f"Existing SL order {order_id} for {symbol} is not open/pending. Status: {details.get('orderstatus')}. Re-placing.")
                        cancel_order(order_id) # Cancel if not open/pending
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                    else: # If details couldn't be fetched, assume it's gone and re-place
                        logger.warning(f"Could not retrieve details for existing SL order {order_id} for {symbol}. Re-placing.")
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                elif not order_id: # No existing SL order, place a new one
                    new_sl_order = place_order(sl_order_params)
                    if new_sl_order:
                        stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                        last_sl_time[symbol] = time.time()


            # Final Exit if conditions met or quantity is zero
            if current_qty <= 0 or exit_position:
                if current_qty > 0 and exit_position: # If there's remaining quantity and an exit condition was met
                    exit_order_params = {
                        "variety": "NORMAL",
                        "tradingsymbol": tradingsymbol,
                        "symboltoken": str(token),
                        "transactiontype": exit_transaction,
                        "exchange": "NSE",
                        "ordertype": "MARKET",
                        "producttype": "INTRADAY",
                        "duration": "DAY",
                        "price": "0.00",
                        "quantity": str(current_qty)
                    }
                    exit_order = place_order(exit_order_params)
                    if exit_order and exit_order.get("status"):
                        completed_final_exit = wait_for_order_completion(exit_order['orderid'])
                        if completed_final_exit:
                            actual_exit_price = float(completed_final_exit['averageprice'])
                            fees = current_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (actual_exit_price - entry_price) * current_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * current_qty
                            net_profit = profit - fees
                            exit_details.append({"price": actual_exit_price, "quantity": current_qty})
                            logger.info(f"Final exit for {symbol}: {current_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                            current_qty = 0 # Mark as fully exited
                        else:
                            logger.error(f"Failed to confirm final exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            # Even if order confirmation fails, assume it was sent for reporting purposes
                            current_qty = 0 # Attempt to clear position state
                    else:
                        logger.error(f"Failed to place final exit order for {symbol}. Current Qty: {current_qty}")
                        # If exit order fails, something is seriously wrong, might need manual intervention

                # Clean up stop loss order if position is closed
                if stop_loss_orders[symbol]:
                    cancel_order(stop_loss_orders[symbol]["order_id"])
                    stop_loss_orders[symbol] = None

                # Record trade summary and clear position state
                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": entry_price,
                    "exit_details": exit_details,
                    "net_pnl": total_net_profit,
                    "profit_layers": profit_layers,
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": exit_reason if exit_reason else "All quantity exited"
                }
                current_positions[symbol] = None # Mark position as closed
                partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                break # Exit the while loop

            time.sleep(1) # Wait for 1 second before next check
    except Exception as e:
        logger.error(f"Fixed target trade loop error for {symbol}: {str(e)}", exc_info=True)
        # Attempt to exit position on critical error
        if current_positions[symbol] and current_qty > 0:
            logger.warning(f"Attempting emergency exit for {symbol} due to error.")
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(current_qty)
            }
            place_order(exit_order_params) # Fire and forget emergency exit
        current_positions[symbol] = None
        stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol):
    global websocket_active, websocket_instances
    if websocket_active.get(symbol):
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP[symbol]
    correlation_id = f"{symbol}_{int(time.time())}"

    # Ensure previous instance is properly closed if it exists and wasn't fully cleaned
    if websocket_instances.get(symbol):
        try:
            websocket_instances[symbol].close()
            logger.info(f"Force-closed stale WebSocket for {symbol} before reconnecting.")
        except Exception as e:
            logger.warning(f"Error force-closing stale WebSocket for {symbol}: {str(e)}")
        websocket_instances[symbol] = None # Clear reference

    ws = SmartWebSocketV2(auth_token=JWT_TOKEN, api_key=API_KEY, client_code=USERNAME, feed_token=FEED_TOKEN)
    websocket_instances[symbol] = ws

    def on_data(ws_instance, message): # Use ws_instance to avoid shadowing
        try:
            if isinstance(message, dict):
                ltp = message.get('ltp')
                if ltp is None and message.get('last_traded_price'):
                    ltp = message['last_traded_price'] / 100.0
            else: # Sometimes messages might come as string, try parsing as JSON
                try:
                    parsed_message = json.loads(message)
                    ltp = parsed_message.get('ltp')
                    if ltp is None and parsed_message.get('last_traded_price'):
                        ltp = parsed_message['last_traded_price'] / 100.0
                except json.JSONDecodeError:
                    logger.warning(f"Could not parse WebSocket message as JSON for {symbol}: {message}")
                    ltp = None

            if ltp:
                ltp_data[symbol] = {"price": float(ltp), "timestamp": time.time()}
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]['price']:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data processing error for {symbol}: {str(e)}", exc_info=True)

    def on_open(ws_instance):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1, # Mode 1 for LTP
                "tokenList": [{"exchangeType": 1, "tokens": [int(token)]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws_instance.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws_instance, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}", exc_info=True)
        with api_lock:
            global api_failures
            api_failures += 1
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

    # Modified on_close to potentially match expected library signature if it changed
    def on_close(ws_instance, *args, **kwargs):
        logger.warning(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs}")
        websocket_active[symbol] = False

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect)
    wst.daemon = True
    wst.start()
    logger.info(f"Attempting to connect WebSocket for {symbol}")

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol):
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = 1.0
    max_attempts = 30 # Number of attempts, not total time
    attempts = 0
    while attempts < max_attempts and not websocket_active.get(symbol): # Stop if WS becomes active
        try:
            ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
            if ltp_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = {"price": float(ltp_response["data"]["ltp"]), "timestamp": time.time()}
                logger.debug(f"API LTP for {symbol} (fallback): {ltp_data[symbol]['price']:.2f}")
                backoff = 1.0 # Reset backoff on success
                attempts = 0 # Reset attempts on success
            else:
                logger.warning(f"LTP API failed for {symbol} (fallback): {ltp_response.get('message', 'No data')}")
                # Re-fetch token only if a token specific error or no data is returned
                token, _ = fetch_symbol_token(symbol) # Try to refresh token
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5) # Max 5 seconds backoff
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}", exc_info=True)
            backoff = min(backoff * 2, 5)
            attempts += 1
        time.sleep(backoff)
    if not websocket_active.get(symbol):
        logger.error(f"Max fallback attempts reached for {symbol} and WebSocket still inactive.")
    else:
        logger.info(f"WebSocket reconnected for {symbol}, stopping fallback price fetch.")


def websocket_watchdog():
    while True:
        time.sleep(WEBSOCKET_RETRY_DELAY) # Check more frequently than 15s if connection is fragile
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                # Closing explicitly before starting new one to clear any lingering states
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close()
                    except Exception:
                        pass
                    websocket_instances[symbol] = None
                start_websocket(symbol) # Attempt to restart WebSocket
                # Also start fallback if websocket fails to connect for a while
                threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()


def ensure_ltp(symbol, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        ltp_info = ltp_data.get(symbol)
        if ltp_info and ltp_info["price"] is not None and (time.time() - ltp_info["timestamp"] < 10): # Check if LTP is recent
            logger.info(f"LTP available for {symbol}: {ltp_info['price']:.2f}")
            return True
        logger.info(f"Waiting for LTP for {symbol}...")
        time.sleep(1)
    logger.error(f"Failed to obtain fresh LTP for {symbol} after {timeout} seconds.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol, trade_type):
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            logger.error(f"Cannot place trade for {symbol}: No LTP available.")
            return None

        quantity = FIXED_QUANTITY
        token, tradingsymbol = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity)
        }

        entry_order = place_order(market_order_params)
        if not entry_order or not entry_order.get("status"): # Check for status in dict
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.5 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": "SELL" if trade_type == "BUY" else "BUY",
            "exchange": "NSE",
            "ordertype": "STOPLOSS_MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity),
            "triggerprice": str(round(stop_price, 1))
        }

        sl_order = place_order(sl_order_params)
        if not sl_order or not sl_order.get("status"):
            logger.critical(f"STOP LOSS ORDER FAILED for {symbol}. Exiting position immediately!")
            # Attempt to exit the entered position if SL fails
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            emergency_exit_order = place_order(exit_params)
            if emergency_exit_order and emergency_exit_order.get("status"):
                logger.info(f"Emergency exit order placed for {symbol} after SL failure.")
            else:
                logger.error(f"CRITICAL: Emergency exit order also failed for {symbol}. Manual intervention required!")
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary():
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"  Type: {summary['trade_type']}\n"
                f"  Entry: {summary['entry_price']:.2f}\n"
                f"  Exits: {exit_prices_str}\n"
                f"  Net P/L: {summary['net_pnl']:.2f}\n"
                f"  Layers: [{profit_layers_str}]\n"
                f"  Reason: {summary['exit_reason']}"
            )
        else:
            logger.info(f"No trade summary available for {symbol}.")


def cleanup(signum=None, frame=None):
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        # Attempt to exit any open positions
        if current_positions[symbol]:
            ltp = ltp_data[symbol]["price"] if ltp_data[symbol]["price"] is not None else current_positions[symbol]["entry_price"]
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if qty > 0 and token and tradingsymbol: # Only try to exit if quantity > 0 and token exists
                logger.info(f"Attempting to exit remaining position for {symbol} (Qty: {qty}) due to script shutdown.")
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order and exit_order.get("status"):
                    completed_exit = wait_for_order_completion(exit_order['orderid'])
                    if completed_exit:
                        actual_exit_price = float(completed_exit['averageprice'])
                        fees = qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                        profit = (actual_exit_price - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - actual_exit_price) * qty
                        net_profit = profit - fees
                        # Update trade_summaries for this forced exit
                        trade_summaries[symbol] = {
                            "trade_type": trade_type,
                            "entry_price": current_positions[symbol]["entry_price"],
                            "exit_details": [{"price": actual_exit_price, "quantity": qty}],
                            "net_pnl": net_profit,
                            "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES], # This might be inaccurate for partial exits
                            "profit_layers_hit": partial_profit_taken[symbol].copy(),
                            "exit_reason": "Script Interrupted (Forced Exit)"
                        }
                        logger.info(f"Exited {trade_type} for {symbol}: {qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted (Forced Exit)")
                    else:
                        logger.error(f"Failed to confirm forced exit order for {symbol}.")
                else:
                    logger.error(f"Failed to place forced exit order for {symbol} on shutdown.")
            else:
                logger.warning(f"No active position or invalid token/tradingsymbol for {symbol} to force exit.")

            # Cancel any open stop-loss orders
            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
                stop_loss_orders[symbol] = None

            current_positions[symbol] = None # Clear position state

        # Close WebSocket connections
        if websocket_instances[symbol]:
            try:
                websocket_instances[symbol].close()
                logger.info(f"Closed WebSocket for {symbol}")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")

    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main():
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        # Check market hours
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()
            return # Exit main if outside hours

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5) # Give some time for websockets to connect and fetch initial LTP

        active_trade_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol): # Only place a trade if no position is active for the symbol
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_trade_threads.append(trade_thread)
                else:
                    logger.error(f"Failed to initiate manual trade for {symbol}. Will not proceed with this symbol.")

        if active_trade_threads:
            logger.info(f"Waiting for {len(active_trade_threads)} trade(s) to complete...")
            for t in active_trade_threads:
                t.join() # Wait for all trade threads to complete
            logger.info("All initiated trades have been completed.")
        else:
            logger.info("No trades were initiated successfully.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup() # Ensure cleanup happens even if an unhandled exception occurs

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'pyotp'

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata'
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 1
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
ltp_data = {symbol: {"price": None, "timestamp": 0} for symbol in SYMBOLS}
current_positions = {symbol: None for symbol in SYMBOLS}
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol, exchange="NSE"):
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params):
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        if isinstance(order, str):
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            error_message = order.get('message', 'Unknown error')
            error_code = order.get('errorcode', 'N/A')
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: Message: {error_message}, Code: {error_code}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}", exc_info=True)
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id):
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id):
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            cancel_response = obj.cancelOrder(order_id, details.get("variety"))
            if cancel_response and cancel_response.get("status"):
                logger.info(f"Cancelled order {order_id}")
                return True
            else:
                logger.warning(f"Failed to cancel order {order_id}: {cancel_response.get('message', 'Unknown error')}")
                return False
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return None

# --- Main Trading Logic ---
def fixed_target_trade(symbol, entry_price, trade_type, quantity, entry_time):
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_trade_loop_time = time.time()
    try:
        while current_positions[symbol]:
            ltp_info = ltp_data.get(symbol)
            ltp = ltp_info["price"] if ltp_info else None

            if ltp is None or (ltp_info and time.time() - ltp_info["timestamp"] > 10):
                logger.warning(f"No fresh LTP for {symbol}. Attempting to fetch via API...")
                for _ in range(5):
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token:
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            fetched_ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = {"price": fetched_ltp, "timestamp": time.time()}
                            ltp = fetched_ltp
                            logger.debug(f"Fetched LTP for {symbol} via API: {ltp:.2f}")
                            break
                        else:
                            logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol} via API: {str(e)}")
                    time.sleep(0.5)

                if ltp is None:
                    if time.time() - start_trade_loop_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol} during trade. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable (Timeout)"
                        break
                    else:
                        logger.warning(f"Still no LTP for {symbol}. Waiting...")
                        time.sleep(1)
                        continue

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            # Exit Conditions
            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            # Partial Profit Booking
            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        profit_qty = int(quantity * profit_percentages[i])
                        if i == NUM_PROFIT_LAYERS - 1:
                            profit_qty = current_qty
                        elif profit_qty == 0 and current_qty > 0:
                            profit_qty = 1 if current_qty >= 1 else 0

                        if profit_qty > 0:
                            exit_order_params = {
                                "variety": "NORMAL",
                                "tradingsymbol": tradingsymbol,
                                "symboltoken": str(token),
                                "transactiontype": exit_transaction,
                                "exchange": "NSE",
                                "ordertype": "MARKET",
                                "producttype": "INTRADAY",
                                "duration": "DAY",
                                "price": "0.00",
                                "quantity": str(profit_qty)
                            }
                            exit_order = place_order(exit_order_params)
                            if exit_order and exit_order["status"]:
                                completed_exit = wait_for_order_completion(exit_order["orderid"])
                                if completed_exit:
                                    actual_exit_price = float(completed_exit['averageprice'])
                                    fees = profit_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                                    profit = (actual_exit_price - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * profit_qty
                                    net_profit = profit - fees
                                    total_net_profit += net_profit
                                    current_qty -= profit_qty
                                    total_profit_taken += 1
                                    exit_details.append({"price": actual_exit_price, "quantity": profit_qty})
                                    partial_profit_taken[symbol][i] = True
                                    tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                                    logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}")
                                    with loss_lock:
                                        if net_profit < 0:
                                            consecutive_losses += 1
                                        else:
                                            consecutive_losses = 0
                                else:
                                    logger.error(f"Failed to confirm profit layer {i+1} exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            else:
                                logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            # Trailing Stop Loss / Breakeven Logic
            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            # Update/Place Stop Loss Order
            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1))
                }

                order_id = stop_loss_orders[symbol].get("order_id") if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        if float(details.get("triggerprice", 0)) != round(sar, 1) or int(details.get("quantity", 0)) != current_qty:
                            try:
                                sl_order_params["orderid"] = order_id
                                obj.modifyOrder(sl_order_params)
                                logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}, Quantity {sl_order_params['quantity']}")
                                last_sl_time[symbol] = time.time()
                            except Exception as e:
                                logger.error(f"SL order modification failed for {symbol}: {str(e)}")
                                cancel_order(order_id)
                                new_sl_order = place_order(sl_order_params)
                                if new_sl_order:
                                    stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                    last_sl_time[symbol] = time.time()
                    elif details:
                        logger.warning(f"Existing SL order {order_id} for {symbol} is not open/pending. Status: {details.get('orderstatus')}. Re-placing.")
                        cancel_order(order_id)
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                    else:
                        logger.warning(f"Could not retrieve details for existing SL order {order_id} for {symbol}. Re-placing.")
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                elif not order_id:
                    new_sl_order = place_order(sl_order_params)
                    if new_sl_order:
                        stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                        last_sl_time[symbol] = time.time()

            # Final Exit
            if current_qty <= 0 or exit_position:
                if current_qty > 0 and exit_position:
                    exit_order_params = {
                        "variety": "NORMAL",
                        "tradingsymbol": tradingsymbol,
                        "symboltoken": str(token),
                        "transactiontype": exit_transaction,
                        "exchange": "NSE",
                        "ordertype": "MARKET",
                        "producttype": "INTRADAY",
                        "duration": "DAY",
                        "price": "0.00",
                        "quantity": str(current_qty)
                    }
                    exit_order = place_order(exit_order_params)
                    if exit_order and exit_order.get("status"):
                        completed_final_exit = wait_for_order_completion(exit_order['orderid'])
                        if completed_final_exit:
                            actual_exit_price = float(completed_final_exit['averageprice'])
                            fees = current_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (actual_exit_price - entry_price) * current_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * current_qty
                            net_profit = profit - fees
                            exit_details.append({"price": actual_exit_price, "quantity": current_qty})
                            logger.info(f"Final exit for {symbol}: {current_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                            current_qty = 0
                        else:
                            logger.error(f"Failed to confirm final exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            current_qty = 0
                    else:
                        logger.error(f"Failed to place final exit order for {symbol}. Current Qty: {current_qty}")
                        current_qty = 0

                    if stop_loss_orders[symbol]:
                        cancel_order(stop_loss_orders[symbol]["order_id"])
                        stop_loss_orders[symbol] = None

                    trade_summaries[symbol] = {
                        "trade_type": trade_type",
                        "entry_price": entry_price,
                        "exit_details": exit_details,
                        "net_pnl": total_net_profit,
                        "profit_layers": profit_layers,
                        "profit_layers_hit": partial_profit_taken[symbol].copy(),
                        "exit_reason": exit_reason if exit_reason else "All quantity removed exit"
                    }
                    current_positions[symbol] = None
                    partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                    break

                time.sleep(1)
    except Exception as e:
        logger.error(f"Fixed target trade loop error for {symbol}: {str(e)}", exc_info=True)
        if current_positions[symbol] and current_qty > 0:
            logger.warning(f"Attempting to exit emergency exit for {symbol} due to {trade_type}")
            token, _ = TOKEN_MAP.get(symbol, (None, ""))
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            exit_transaction_type = "exit"
            exit_order_params = {
                "variety": "normal",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INDTRADAY",
                "duration": "DAY",
                "price": str(current_qty),
                "quantity": str(token)
            }
            place_order = place_order(exit_order_params)
            exit_order(order_id)
            current_positions[symbol] = None
            stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol):
    global websocket_active, websocket_instances
    if websocket_active[symbol]:
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    correlation_id = f"{symbol}_{int(time.time())})".format(int(time.time()))

    if websocket_instances[symbol]:
        try:
            websocket_instances[symbol].connection_close_connection()
            logger.info(f"Closed stale WebSocket for {symbol} before reconnecting")
        except Exception as e:
            logger.warning(f"Error closing stale WebSocket for {symbol}: {str(e)}")
        websocket_instances[symbol] = None

    ws = SmartWebSocketV2(
        auth_token=JWT_TOKEN,
        api_key=API_KEY,
        client_code=USERNAME,
        feed_token=FEED_TOKEN
    )
    websocket_instances[symbol] = ws

    def on_data(ws_instance, message):
        try:
            if isinstance(message, dict):
                ltp_price = message.get('ltp')
                if ltp is None and message.get('last_traded_price'):
                    ltp = ltp_message['last_traded_price'] / 100.0
            else:
                try:
                    parsed_message = json.loads(message)
                    ltp = parsed_message.get('ltp')
                    if ltp is None and parsed_message.get('last_traded_price'):
                        ltp = parsed_message['last_traded_price'] / 100.0
                except json.JSONDecodeError:
                    logger.warning(f"Could not parse WebSocket message as JSON for {symbol}: {message}")
                    ltp = None

            if ltp:
                ltp_data[symbol] = {"price": float(ltp), "timestamp": time.time()}
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]['price']:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data processing error for {symbol}: {str(e)}", exc_info=True)

    def on_open(ws_instance):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1,
                "tokenList": [{"exchangeType": 1, "tokens": [int(token)]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws_instance.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws_instance, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}", exc_info=True)
        with api_lock:
            global api_failures
            api_key += 1
            if api_key >= MAX_API_TOKEN:
                logger.critical(f"Maximum API failures reached. Exiting.")
                sys.exit(1)

    def on_close(ws_instance, *args, **kwargs):
        logger.info(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs})")
        websocket_active[symbol] = False

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect)
    wst.daemon = target=True
    wst.start()
    logger.info(f"Started WebSocket thread for {symbol}")

    @retry(stop_max_attempt_number=3, wait_for_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol):
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, str(None))))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = token1.0
    max_attempts = 30
    attempts ಠ_ಠ= 0
    while attempts < max_attempts and not websocket_active[symbol]:
        try:
            ltp_response = obj.ltp(symbol + "-NSE", symbol + "-EQ", token)
            if ltp_data_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = {"price": float(ltp_response["data"]["ltp"]), "timestamp": time.time()}
                logger.debug(f"API LTP for {symbol} (fallback): {ltp_data[symbol]['price']:.2f}")
                backoff = float(1.0)
                attempts = 0
            else:
                logger.warning(f"LTP API LTP failed for {symbol} for {symbol} (fallback): {str(e)}")
                token, _ = fetch_symbol(symbol)
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5)
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}", exc_info=True)
            backoff = min(backoff * 2, token)
            attempts += 1
        time.sleep(backoff)
    if not websocket_active[symbol]:
        logger.error(f"Max fallback attempts reached for {symbol} and WebSocket still inactive.")
    else:
        logger.info(f"WebSocket reconnected for {symbol}, stopping fallback price fetch.")

def websocket_watchdog():
    while True:
        time.sleep(WEBSOCKET_RETRY_DELAY)
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close_connection()
                        logger.info(f"Closed stale WebSocket for {symbol} before reconnecting.")
                    except Exception as e:
                        logger.warning(f"Error closing stale WebSocket for {symbol}: {str(e)}")
                    websocket_instances[symbol] = None
                start_websocket(symbol)
                threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()

def ensure_ltp(symbol, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        ltp_info = ltp_data.get(symbol)
        if ltp_info and ltp_info["price"] is not None and (time.time() - ltp_info["timestamp"] < 10):
            logger.info(f"LTP available for {symbol}: {ltp_info['price']:.2f}")
            return True
        logger.info(f"Waiting for LTP for {symbol}...")
        time.sleep(1)
    logger.error(f"Timeout waiting for LTP for {symbol}.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol, trade_type):
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            logger.error(f"Cannot place trade for {symbol}: No LTP available.")
            return None
        trade_type = None
        quantity = FIXED_QUANTITY
        token, _ = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": symbol + "-EQ",
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": str(quantity),
            "quantity": "0.0"
        }

        entry_order = place_order(market_order_params)
        if not entry_order or not entry_order.get("status"):
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.0 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": symbol + "-EQ",
            "symboltoken": str(token),
            "exit": "SELL" if trade_type == "BUY" else "EXIT",
            "exchange": "NSE",
            "profit_type": str(stop_price),
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": str(0.0),
            "quantity": str(quantity),
            "trigger_price": str(round(stop_price, price1))
        }

        sl_order = place_order(sl_order_params)
        order_id = order if sl_order else None
        if not sl_order or not sl_order.get("status"):
            logger.critical(f"STOP LOSS ORDER FAILED for {order_id}. Exiting position immediately!")
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            emergency_exit_order = place_order(exit_params)
            if emergency_exit_order and emergency_exit_order.get("status"):
                logger.info(f"Emergency exit order placed for {symbol} after SL failure.")
            else:
                logger.error(f"CRITICAL: Emergency exit order also failed for {symbol}. Manual intervention required!")
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary():
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"  Type: {summary['trade_type']}\n"
                f"  Entry: {summary['entry_price']:.2f}\n"
                f"  Exits: {exit_prices_str}\n"
                f"  Net P/L: {summary['net_pnl']:.2f}\n"
                f"  Layers: [{profit_layers_str}]\n"
                f"  Reason: {summary['exit_reason']}"
            )
        else:
            logger.info(f"No trade summary available for {symbol}.")

def cleanup(signum=None, frame=None):
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        if current_positions[symbol]:
            ltp = ltp_data[symbol]["price"] if ltp_data[symbol]["price"] is not None else current_positions[symbol]["entry_price"]
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if qty > 0 and token and tradingsymbol:
                logger.info(f"Attempting to exit remaining position for {symbol} (Qty: {qty}) due to script shutdown.")
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order and exit_order.get("status"):
                    completed_exit = wait_for_order_completion(exit_order['orderid'])
                    if completed_exit:
                        actual_exit_price = float(completed_exit['averageprice'])
                        fees = qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                        profit = (actual_exit_price - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - actual_exit_price) * qty
                        net_profit = profit - fees
                        trade_summaries[symbol] = {
                            "trade_type": trade_type,
                            "entry_price": current_positions[symbol]["entry_price"],
                            "exit_details": [{"price": actual_exit_price, "quantity": qty}],
                            "net_pnl": net_profit,
                            "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES],
                            "profit_layers_hit": partial_profit_taken[symbol].copy(),
                            "exit_reason": "Script Interrupted (Forced Exit)"
                        }
                        logger.info(f"Exited {trade_type} for {symbol}: {qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted (Forced Exit)")
                    else:
                        logger.error(f"Failed to confirm forced exit order for {symbol}.")
                else:
                    logger.error(f"Failed to place forced exit order for {symbol} on shutdown.")
            else:
                logger.warning(f"No active position or invalid token/tradingsymbol for {symbol} to force exit.")

            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
                stop_loss_orders[symbol] = None

            current_positions[symbol] = None

        if websocket_instances[symbol]:
            try:
                if hasattr(websocket_instances[symbol], 'close_connection'):
                    websocket_instances[symbol].close_connection()
                    logger.info(f"Closed WebSocket for {symbol}")
                else:
                    logger.warning(f"No close_connection method for WebSocket {symbol}. Skipping closure.")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")
            websocket_instances[symbol] = None

    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main():
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()
            return

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5)

        active_trade_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol):
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_trade_threads.append(trade_thread)
                else:
                    logger.error(f"Failed to initiate manual trade for {symbol}.")

        if active_trade_threads:
            logger.info(f"Waiting for {len(active_trade_threads)} trade(s) to complete...")
            for t in active_trade_threads:
                t.join()
            logger.info("All initiated trades have been completed.")
        else:
            logger.info("No trades were initiated successfully.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup()

if __name__ == "__main__":
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 656)

In [ ]:
import pyotp
from SmartApi.smartConnect import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import logzero
from logzero import logger
from retrying import retry
import time
from datetime import datetime
import threading
import json
import requests
import pytz
import os
import signal
import sys
from typing import Dict, List, Optional, Tuple, Union

# Set IST timezone
os.environ['TZ'] = 'Asia/Kolkata'
try:
    time.tzset()
except AttributeError:
    pass

# --- Configuration ---
API_KEY = os.getenv("API_KEY", "OBpBVquO")
USERNAME = os.getenv("USERNAME", "AAAN189703")
PASSWORD = os.getenv("PASSWORD", "5350")
TOTP_SECRET = os.getenv("TOTP_SECRET", "X4BL4SW4CHW566VGREUMMNA5HQ")

# --- Trading Parameters ---
SYMBOLS = ["MOTHERSON"]
MAX_API_FAILURES = 5
MIN_MODIFY_INTERVAL = 5
FIXED_TARGET_POINTS = 0.5
NUM_PROFIT_LAYERS = 6
PROFIT_DISTANCES = [0.10, 0.20, 0.30, 0.40, 0.45, 0.50]
TIGHTENING_FACTOR_MIN = 0.5
TIGHTENING_FACTOR_STEP = 0.1
FIXED_QUANTITY = 6
MANUAL_TRADE_TYPE = "BUY"

# --- Fees and Slippage ---
BROKERAGE_FEE = 0.00236
STT_CTT = 0.00025
EXCHANGE_FEE = 0.0000324
SLIPPAGE = 0.001

# --- Timing and Retries ---
EXIT_TIME_SECONDS = 22500
WEBSOCKET_RETRY_DELAY = 5
LTP_TIMEOUT = 30

# --- Configuration Validation ---
if len(PROFIT_DISTANCES) != NUM_PROFIT_LAYERS:
    logger.error(f"PROFIT_DISTANCES length ({len(PROFIT_DISTANCES)}) does not match NUM_PROFIT_LAYERS ({NUM_PROFIT_LAYERS})")
    sys.exit(1)
if PROFIT_DISTANCES[-1] != FIXED_TARGET_POINTS:
    logger.error(f"Last PROFIT_DISTANCE ({PROFIT_DISTANCES[-1]}) does not match FIXED_TARGET_POINTS ({FIXED_TARGET_POINTS})")
    sys.exit(1)

# --- Global State and Locks ---
TOKEN_MAP = {}
FALLBACK_TOKEN_MAP = {"MOTHERSON": ("4204", "MOTHERSON-EQ")}
logzero.logfile("trading_log_fixed_target_real.txt", maxBytes=1e6, backupCount=5)
logger.info("Trading script started with fixed target (real trading)")

api_lock = threading.Lock()
loss_lock = threading.Lock()
daily_trades = 0
consecutive_losses = 0
api_failures = 0
ltp_data = {symbol: {"price": None, "timestamp": 0} for symbol in SYMBOLS}
current_positions = {symbol: None for symbol in SYMBOLS}
stop_loss_orders = {symbol: None for symbol in SYMBOLS}
partial_profit_taken = {symbol: [False] * NUM_PROFIT_LAYERS for symbol in SYMBOLS}
last_sl_time = {symbol: 0 for symbol in SYMBOLS}
trade_summaries = {symbol: {} for symbol in SYMBOLS}
websocket_active = {symbol: False for symbol in SYMBOLS}
websocket_instances = {symbol: None for symbol in SYMBOLS}

# --- Initial Setup ---
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fetch_symbol_token(symbol: str, exchange: str = "NSE") -> Tuple[Optional[str], Optional[str]]:
    try:
        url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        instruments = response.json()
        for instrument in instruments:
            if instrument.get("symbol", "").upper() == f"{symbol.upper()}-EQ":
                token = instrument.get("token")
                tradingsymbol = instrument.get("symbol")
                logger.info(f"Added token {token} for {tradingsymbol}")
                return token, tradingsymbol
        logger.warning(f"No token found for {symbol}. Using fallback.")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))
    except Exception as e:
        logger.error(f"Failed to fetch token for {symbol}: {str(e)}")
        return FALLBACK_TOKEN_MAP.get(symbol, (None, ""))

for symbol in SYMBOLS:
    token, tradingsymbol = fetch_symbol_token(symbol)
    if token and tradingsymbol:
        TOKEN_MAP[symbol] = (token, tradingsymbol)
    else:
        logger.error(f"No token for {symbol}. Exiting.")
        sys.exit(1)
logger.debug(f"Initialized TOKEN_MAP: {TOKEN_MAP}")

# --- API Connection ---
obj = SmartConnect(api_key=API_KEY)
totp = pyotp.TOTP(TOTP_SECRET)
data = obj.generateSession(USERNAME, PASSWORD, totp.now())
if not data.get("status"):
    logger.error(f"Login failed: {data.get('message')}")
    sys.exit(1)
logger.info("Login successful")
FEED_TOKEN = data["data"]["feedToken"]
JWT_TOKEN = data["data"]["jwtToken"]
time.sleep(2)

# --- API Wrapper Functions ---
@retry(stop_max_attempt_number=3, wait_fixed=2000)
def place_order(order_params: Dict) -> Optional[Dict]:
    global api_failures
    try:
        order = obj.placeOrder(order_params)
        if isinstance(order, str):
            logger.info(f"Placed {order_params['transactiontype']} order for {order_params['tradingsymbol']}: Order ID {order}")
            with api_lock:
                api_failures = 0
            return {"status": True, "orderid": order}
        elif isinstance(order, dict) and not order.get("status"):
            error_message = order.get('message', 'Unknown error')
            error_code = order.get('errorcode', 'N/A')
            logger.error(f"Order placement failed for {order_params['tradingsymbol']}: Message: {error_message}, Code: {error_code}")
            with api_lock:
                api_failures += 1
            return None
        else:
            logger.error(f"Unexpected order response for {order_params['tradingsymbol']}: {order}")
            with api_lock:
                api_failures += 1
            return None
    except Exception as e:
        logger.error(f"Order placement exception for {order_params['tradingsymbol']}: {str(e)}", exc_info=True)
        with api_lock:
            api_failures += 1
        return None
    finally:
        with api_lock:
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_order_details(order_id: str) -> Optional[Dict]:
    try:
        order_book = obj.orderBook()
        if order_book and order_book.get("status"):
            for order in order_book.get("data", []):
                if order.get("orderid") == order_id:
                    return order
        return None
    except Exception as e:
        logger.error(f"Failed to fetch order details for {order_id}: {str(e)}")
        return None

def wait_for_order_completion(order_id: str, timeout: int = 30) -> Optional[Dict]:
    start_time = time.time()
    while time.time() - start_time < timeout:
        details = get_order_details(order_id)
        if details:
            status = details.get("orderstatus", "").upper()
            if status == "COMPLETE":
                avg_price = float(details.get('averageprice', 0))
                logger.info(f"Order {order_id} completed at average price: {avg_price:.2f}")
                return details
            if status in ["REJECTED", "CANCELLED"]:
                logger.error(f"Order {order_id} failed with status: {status}. Reason: {details.get('text')}")
                return None
        time.sleep(1)
    logger.error(f"Timeout waiting for order {order_id} completion.")
    return None

@retry(stop_max_attempt_number=3, wait_fixed=1000)
def cancel_order(order_id: str) -> bool:
    try:
        details = get_order_details(order_id)
        if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
            cancel_response = obj.cancelOrder(order_id, details.get("variety"))
            if cancel_response and cancel_response.get("status"):
                logger.info(f"Cancelled order {order_id}")
                return True
            else:
                logger.warning(f"Failed to cancel order {order_id}: {cancel_response.get('message', 'Unknown error')}")
                return False
        elif details:
            logger.warning(f"Cannot cancel order {order_id}, status is {details.get('orderstatus')}")
            return False
        else:
            logger.warning(f"Could not retrieve details for order {order_id} to cancel.")
            return False
    except Exception as e:
        logger.error(f"Failed to cancel order {order_id}: {str(e)}")
        return False

# --- Main Trading Logic ---
def fixed_target_trade(symbol: str, entry_price: float, trade_type: str, quantity: int, entry_time: float) -> None:
    global consecutive_losses, trade_summaries, stop_loss_orders
    stop_loss_factor = 0.5
    profit_percentages = [1.0 / NUM_PROFIT_LAYERS for _ in range(NUM_PROFIT_LAYERS)]
    tightening_factor = 1.0

    if trade_type == "BUY":
        initial_stop = entry_price - stop_loss_factor
        exit_transaction = "SELL"
        profit_layers = [entry_price + d for d in PROFIT_DISTANCES]
    else:
        initial_stop = entry_price + stop_loss_factor
        exit_transaction = "BUY"
        profit_layers = [entry_price - d for d in PROFIT_DISTANCES]

    sar = initial_stop
    breakeven_set = False
    total_profit_taken = 0
    current_qty = quantity
    exit_details = []
    total_net_profit = 0
    profit_layers_str = f"[{', '.join([f'{p:.2f}' for p in profit_layers])}]"
    logger.info(f"Initialized {trade_type} for {symbol}: Entry {entry_price:.2f}, TP {entry_price + FIXED_TARGET_POINTS:.2f}, SL {initial_stop:.2f}, Profit Layers {profit_layers_str}")

    start_trade_loop_time = time.time()
    try:
        while current_positions[symbol]:
            ltp_info = ltp_data.get(symbol)
            ltp = ltp_info["price"] if ltp_info else None

            if ltp is None or (ltp_info and time.time() - ltp_info["timestamp"] > 10):
                logger.warning(f"No fresh LTP for {symbol}. Attempting to fetch via API...")
                for _ in range(5):
                    try:
                        token, _ = TOKEN_MAP.get(symbol, (None, ""))
                        if not token:
                            token, _ = fetch_symbol_token(symbol)
                            if token:
                                TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                            else:
                                logger.error(f"No valid token for {symbol}")
                                break
                        ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
                        if ltp_response["status"] and "data" in ltp_response:
                            fetched_ltp = float(ltp_response["data"]["ltp"])
                            ltp_data[symbol] = {"price": fetched_ltp, "timestamp": time.time()}
                            ltp = fetched_ltp
                            logger.debug(f"Fetched LTP for {symbol} via API: {ltp:.2f}")
                            break
                        else:
                            logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                    except Exception as e:
                        logger.error(f"LTP fetch error for {symbol} via API: {str(e)}")
                    time.sleep(0.5)

                if ltp is None:
                    if time.time() - start_trade_loop_time > LTP_TIMEOUT:
                        logger.error(f"LTP timeout for {symbol} during trade. Exiting position.")
                        exit_position = True
                        exit_reason = "LTP Unavailable (Timeout)"
                        break
                    else:
                        logger.warning(f"Still no LTP for {symbol}. Waiting...")
                        time.sleep(1)
                        continue

            logger.debug(f"Checking {symbol} ({trade_type}): LTP={ltp:.2f}, TP={entry_price + FIXED_TARGET_POINTS:.2f}, SL={sar:.2f}")

            exit_position = False
            exit_reason = ""
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            # Exit Conditions
            if trade_type == "BUY" and ltp >= entry_price + FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif trade_type == "SELL" and ltp <= entry_price - FIXED_TARGET_POINTS:
                exit_position = True
                exit_reason = "Target Price Hit"
            elif time.time() - entry_time > EXIT_TIME_SECONDS:
                exit_position = True
                exit_reason = "Time Exit"
            elif trade_type == "BUY" and ltp <= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"
            elif trade_type == "SELL" and ltp >= sar:
                exit_position = True
                exit_reason = "Stop Loss Hit"

            # Partial Profit Booking
            for i in range(NUM_PROFIT_LAYERS):
                if not partial_profit_taken[symbol][i] and current_qty > 0:
                    profit_price = profit_layers[i]
                    condition = ltp >= profit_price if trade_type == "BUY" else ltp <= profit_price
                    if condition:
                        profit_qty = int(quantity * profit_percentages[i])
                        if i == NUM_PROFIT_LAYERS - 1:
                            profit_qty = current_qty
                        elif profit_qty == 0 and current_qty > 0:
                            profit_qty = 1 if current_qty >= 1 else 0

                        if profit_qty > 0:
                            exit_order_params = {
                                "variety": "NORMAL",
                                "tradingsymbol": tradingsymbol,
                                "symboltoken": str(token),
                                "transactiontype": exit_transaction,
                                "exchange": "NSE",
                                "ordertype": "MARKET",
                                "producttype": "INTRADAY",
                                "duration": "DAY",
                                "price": "0.00",
                                "quantity": str(profit_qty)
                            }
                            exit_order = place_order(exit_order_params)
                            if exit_order and exit_order["status"]:
                                completed_exit = wait_for_order_completion(exit_order["orderid"])
                                if completed_exit:
                                    actual_exit_price = float(completed_exit['averageprice'])
                                    fees = profit_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                                    profit = (actual_exit_price - entry_price) * profit_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * profit_qty
                                    net_profit = profit - fees
                                    total_net_profit += net_profit
                                    current_qty -= profit_qty
                                    total_profit_taken += 1
                                    exit_details.append({"price": actual_exit_price, "quantity": profit_qty})
                                    partial_profit_taken[symbol][i] = True
                                    tightening_factor = max(TIGHTENING_FACTOR_MIN, tightening_factor - TIGHTENING_FACTOR_STEP)
                                    logger.info(f"Booked profit layer {i+1} for {symbol}: {profit_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}")
                                    with loss_lock:
                                        if net_profit < 0:
                                            consecutive_losses += 1
                                        else:
                                            consecutive_losses = 0
                                else:
                                    logger.error(f"Failed to confirm profit layer {i+1} exit order for {symbol}. Order ID: {exit_order['orderid']}")
                            else:
                                logger.error(f"Failed to place profit layer {i+1} exit order for {symbol}")

            # Trailing Stop Loss / Breakeven Logic
            if total_profit_taken >= 3 and not breakeven_set:
                sar = entry_price
                breakeven_set = True
                logger.info(f"Set breakeven for {symbol}: SL {sar:.2f}")

            # Update/Place Stop Loss Order
            if current_qty > 0 and not exit_position:
                sl_order_params = {
                    "variety": "STOPLOSS",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "STOPLOSS_MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(current_qty),
                    "triggerprice": str(round(sar, 1))
                }

                order_id = stop_loss_orders[symbol]["order_id"] if stop_loss_orders[symbol] else None
                if order_id and time.time() - last_sl_time[symbol] >= MIN_MODIFY_INTERVAL:
                    details = get_order_details(order_id)
                    if details and details.get("orderstatus", "").upper() in ["OPEN", "TRIGGER PENDING"]:
                        if float(details.get("triggerprice", 0)) != round(sar, 1) or int(details.get("quantity", 0)) != current_qty:
                            try:
                                sl_order_params["orderid"] = order_id
                                obj.modifyOrder(sl_order_params)
                                logger.info(f"Modified SL order for {symbol}: Trigger {sl_order_params['triggerprice']}, Quantity {sl_order_params['quantity']}")
                                last_sl_time[symbol] = time.time()
                            except Exception as e:
                                logger.error(f"SL order modification failed for {symbol}: {str(e)}")
                                cancel_order(order_id)
                                new_sl_order = place_order(sl_order_params)
                                if new_sl_order:
                                    stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                                    last_sl_time[symbol] = time.time()
                    elif details:
                        logger.warning(f"Existing SL order {order_id} for {symbol} is not open/pending. Status: {details.get('orderstatus')}. Re-placing.")
                        cancel_order(order_id)
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                    else:
                        logger.warning(f"Could not retrieve details for existing SL order {order_id} for {symbol}. Re-placing.")
                        new_sl_order = place_order(sl_order_params)
                        if new_sl_order:
                            stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                            last_sl_time[symbol] = time.time()
                elif not order_id:
                    new_sl_order = place_order(sl_order_params)
                    if new_sl_order:
                        stop_loss_orders[symbol] = {"order_id": new_sl_order["orderid"]}
                        last_sl_time[symbol] = time.time()

            # Final Exit
            if current_qty <= 0 or exit_position:
                if current_qty > 0 and exit_position:
                    exit_order_params = {
                        "variety": "NORMAL",
                        "tradingsymbol": tradingsymbol,
                        "symboltoken": str(token),
                        "transactiontype": exit_transaction,
                        "exchange": "NSE",
                        "ordertype": "MARKET",
                        "producttype": "INTRADAY",
                        "duration": "DAY",
                        "price": "0.00",
                        "quantity": str(current_qty)
                    }
                    exit_order = place_order(exit_order_params)
                    if exit_order and exit_order.get("status"):
                        completed_final_exit = wait_for_order_completion(exit_order['orderid'])
                        if completed_final_exit:
                            actual_exit_price = float(completed_final_exit['averageprice'])
                            fees = current_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                            profit = (actual_exit_price - entry_price) * current_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * current_qty
                            net_profit = profit - fees
                            total_net_profit += net_profit
                            exit_details.append({"price": actual_exit_price, "quantity": current_qty})
                            logger.info(f"Final exit for {symbol}: {current_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: {exit_reason}")
                            with loss_lock:
                                if net_profit < 0:
                                    consecutive_losses += 1
                                else:
                                    consecutive_losses = 0
                        else:
                            logger.error(f"Failed to confirm final exit order for {symbol}. Order ID: {exit_order['orderid']}")
                        current_qty = 0
                    else:
                        logger.error(f"Failed to place final exit order for {symbol}. Current Qty: {current_qty}")
                        current_qty = 0

                if stop_loss_orders[symbol]:
                    cancel_order(stop_loss_orders[symbol]["order_id"])
                    stop_loss_orders[symbol] = None

                trade_summaries[symbol] = {
                    "trade_type": trade_type,
                    "entry_price": entry_price,
                    "exit_details": exit_details,
                    "net_pnl": total_net_profit,
                    "profit_layers": profit_layers,
                    "profit_layers_hit": partial_profit_taken[symbol].copy(),
                    "exit_reason": exit_reason if exit_reason else "All quantity removed"
                }
                current_positions[symbol] = None
                partial_profit_taken[symbol] = [False] * NUM_PROFIT_LAYERS
                break

            time.sleep(1)
    except Exception as e:
        logger.error(f"Fixed target trade loop error for {symbol}: {str(e)}", exc_info=True)
        if current_positions[symbol] and current_qty > 0:
            logger.warning(f"Attempting emergency exit for {symbol} due to error")
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            exit_order_params = {
                "variety": "NORMAL",
                "tradingsymbol": tradingsymbol,
                "symboltoken": str(token),
                "transactiontype": exit_transaction,
                "exchange": "NSE",
                "ordertype": "MARKET",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": "0.00",
                "quantity": str(current_qty)
            }
            exit_order = place_order(exit_order_params)
            if exit_order and exit_order.get("status"):
                completed_exit = wait_for_order_completion(exit_order['orderid'])
                if completed_exit:
                    actual_exit_price = float(completed_exit['averageprice'])
                    fees = current_qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                    profit = (actual_exit_price - entry_price) * current_qty if trade_type == "BUY" else (entry_price - actual_exit_price) * current_qty
                    net_profit = profit - fees
                    total_net_profit += net_profit
                    exit_details.append({"price": actual_exit_price, "quantity": current_qty})
                    trade_summaries[symbol] = {
                        "trade_type": trade_type,
                        "entry_price": entry_price,
                        "exit_details": exit_details,
                        "net_pnl": total_net_profit,
                        "profit_layers": profit_layers,
                        "profit_layers_hit": partial_profit_taken[symbol].copy(),
                        "exit_reason": "Emergency Exit due to Error"
                    }
                    logger.info(f"Emergency exit for {symbol}: {current_qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}")
                else:
                    logger.error(f"Failed to confirm emergency exit order for {symbol}")
            else:
                logger.error(f"Failed to place emergency exit order for {symbol}")
            current_positions[symbol] = None
            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
                stop_loss_orders[symbol] = None

# --- WebSocket and Price Feed ---
def start_websocket(symbol: str) -> None:
    global websocket_active, websocket_instances
    if websocket_active[symbol]:
        logger.info(f"WebSocket already active for {symbol}")
        return

    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    correlation_id = f"{symbol}_{int(time.time())}"

    if websocket_instances[symbol]:
        try:
            websocket_instances[symbol].close_connection()
            logger.info(f"Closed stale WebSocket for {symbol} before reconnecting")
        except Exception as e:
            logger.warning(f"Error closing stale WebSocket for {symbol}: {str(e)}")
        websocket_instances[symbol] = None

    ws = SmartWebSocketV2(
        auth_token=JWT_TOKEN,
        api_key=API_KEY,
        client_code=USERNAME,
        feed_token=FEED_TOKEN
    )
    websocket_instances[symbol] = ws

    def on_data(ws_instance, message):
        try:
            if isinstance(message, dict):
                ltp = message.get('ltp')
                if ltp is None and message.get('last_traded_price'):
                    ltp = message['last_traded_price'] / 100.0
            else:
                try:
                    parsed_message = json.loads(message)
                    ltp = parsed_message.get('ltp')
                    if ltp is None and parsed_message.get('last_traded_price'):
                        ltp = parsed_message['last_traded_price'] / 100.0
                except json.JSONDecodeError:
                    logger.warning(f"Could not parse WebSocket message as JSON for {symbol}: {message}")
                    ltp = None

            if ltp is not None:
                ltp_data[symbol] = {"price": float(ltp), "timestamp": time.time()}
                logger.debug(f"WebSocket LTP for {symbol}: {ltp_data[symbol]['price']:.2f}")
        except Exception as e:
            logger.error(f"WebSocket data processing error for {symbol}: {str(e)}", exc_info=True)

    def on_open(ws_instance):
        logger.info(f"WebSocket opened for {symbol}")
        websocket_active[symbol] = True
        subscribe_data = {
            "correlationID": correlation_id,
            "action": 1,
            "params": {
                "mode": 1,
                "tokenList": [{"exchangeType": 1, "tokens": [token]}]
            }
        }
        logger.debug(f"Sending subscription for {symbol}: {subscribe_data}")
        ws_instance.send(json.dumps(subscribe_data))
        logger.info(f"Subscribed to {symbol} with token {token}")

    def on_error(ws_instance, error):
        logger.error(f"WebSocket error for {symbol}: {str(error)}", exc_info=True)
        with api_lock:
            global api_failures
            api_failures += 1
            if api_failures >= MAX_API_FAILURES:
                logger.critical("Maximum API failures reached. Exiting.")
                sys.exit(1)

    def on_close(ws_instance, *args, **kwargs):
        logger.info(f"WebSocket closed for {symbol}. Args: {args}, Kwargs: {kwargs}")
        websocket_active[symbol] = False

    ws.on_open = on_open
    ws.on_data = on_data
    ws.on_error = on_error
    ws.on_close = on_close

    wst = threading.Thread(target=ws.connect, daemon=True)
    wst.start()
    logger.info(f"Started WebSocket thread for {symbol}")

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=10000)
def fallback_price_fetch(symbol: str) -> None:
    logger.info(f"Starting fallback price fetch for {symbol}")
    token, _ = TOKEN_MAP.get(symbol, (None, ""))
    if not token:
        logger.error(f"No token for {symbol} in fallback")
        return
    backoff = 1.0
    max_attempts = 30
    attempts = 0
    while attempts < max_attempts and not websocket_active[symbol]:
        try:
            ltp_response = obj.ltpData("NSE", symbol + "-EQ", token)
            if ltp_response["status"] and "data" in ltp_response:
                ltp_data[symbol] = {"price": float(ltp_response["data"]["ltp"]), "timestamp": time.time()}
                logger.debug(f"API LTP for {symbol} (fallback): {ltp_data[symbol]['price']:.2f}")
                backoff = 1.0
                attempts = 0
            else:
                logger.warning(f"LTP API failed for {symbol}: {ltp_response.get('message', 'No data')}")
                token, _ = fetch_symbol_token(symbol)
                if token:
                    TOKEN_MAP[symbol] = (token, symbol + "-EQ")
                backoff = min(backoff * 2, 5)
                attempts += 1
        except Exception as e:
            logger.error(f"Fallback price fetch error for {symbol}: {str(e)}", exc_info=True)
            backoff = min(backoff * 2, 5)
            attempts += 1
        time.sleep(backoff)
    if not websocket_active[symbol]:
        logger.error(f"Max fallback attempts reached for {symbol} and WebSocket still inactive.")
    else:
        logger.info(f"WebSocket reconnected for {symbol}, stopping fallback price fetch.")

def websocket_watchdog() -> None:
    while True:
        time.sleep(WEBSOCKET_RETRY_DELAY)
        for symbol in SYMBOLS:
            if not websocket_active.get(symbol):
                logger.warning(f"Watchdog: WebSocket for {symbol} is inactive. Attempting to restart.")
                if websocket_instances[symbol]:
                    try:
                        websocket_instances[symbol].close_connection()
                        logger.info(f"Closed stale WebSocket for {symbol} before reconnecting.")
                    except Exception as e:
                        logger.warning(f"Error closing stale WebSocket for {symbol}: {str(e)}")
                    websocket_instances[symbol] = None
                start_websocket(symbol)
                threading.Thread(target=fallback_price_fetch, args=(symbol,), daemon=True).start()

def ensure_ltp(symbol: str, timeout: int = 30) -> bool:
    start_time = time.time()
    while time.time() - start_time < timeout:
        ltp_info = ltp_data.get(symbol)
        if ltp_info and ltp_info["price"] is not None and (time.time() - ltp_info["timestamp"] < 10):
            logger.info(f"LTP available for {symbol}: {ltp_info['price']:.2f}")
            return True
        logger.info(f"Waiting for LTP for {symbol}...")
        time.sleep(1)
    logger.error(f"Timeout waiting for LTP for {symbol}.")
    return False

# --- Trade Execution ---
def place_manual_trade(symbol: str, trade_type: str) -> Optional[threading.Thread]:
    global daily_trades
    try:
        if not ensure_ltp(symbol):
            logger.error(f"Cannot place trade for {symbol}: No LTP available.")
            return None
        quantity = FIXED_QUANTITY
        token, tradingsymbol = TOKEN_MAP[symbol]

        market_order_params = {
            "variety": "NORMAL",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": trade_type,
            "exchange": "NSE",
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity)
        }

        entry_order = place_order(market_order_params)
        if not entry_order or not entry_order.get("status"):
            logger.error(f"Entry order failed for {symbol}. Aborting trade.")
            return None

        completed_entry = wait_for_order_completion(entry_order['orderid'])
        if not completed_entry:
            logger.error(f"Could not confirm entry order completion for {symbol}. Aborting.")
            return None

        entry_price = float(completed_entry['averageprice'])
        stop_price = entry_price - 0.5 if trade_type == "BUY" else entry_price + 0.5

        sl_order_params = {
            "variety": "STOPLOSS",
            "tradingsymbol": tradingsymbol,
            "symboltoken": str(token),
            "transactiontype": "SELL" if trade_type == "BUY" else "BUY",
            "exchange": "NSE",
            "ordertype": "STOPLOSS_MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0.00",
            "quantity": str(quantity),
            "triggerprice": str(round(stop_price, 1))
        }

        sl_order = place_order(sl_order_params)
        if not sl_order or not sl_order.get("status"):
            logger.critical(f"Stop loss order failed for {symbol}. Exiting position immediately!")
            exit_params = market_order_params.copy()
            exit_params["transactiontype"] = "SELL" if trade_type == "BUY" else "BUY"
            emergency_exit_order = place_order(exit_params)
            if emergency_exit_order and emergency_exit_order.get("status"):
                logger.info(f"Emergency exit order placed for {symbol} after SL failure.")
            else:
                logger.error(f"CRITICAL: Emergency exit order also failed for {symbol}. Manual intervention required!")
            return None

        current_positions[symbol] = {
            "type": trade_type,
            "entry_price": entry_price,
            "quantity": quantity,
            "entry_time": time.time()
        }
        stop_loss_orders[symbol] = {"order_id": sl_order['orderid']}
        daily_trades += 1

        logger.info(f"Successfully placed {trade_type} for {symbol}: {quantity} @ {entry_price:.2f}, SL: {stop_price:.2f}")

        trade_thread = threading.Thread(
            target=fixed_target_trade,
            args=(symbol, entry_price, trade_type, quantity, time.time()),
            daemon=True
        )
        return trade_thread

    except Exception as e:
        logger.error(f"Error placing manual trade for {symbol}: {str(e)}", exc_info=True)
        return None

def log_final_summary() -> None:
    for symbol in SYMBOLS:
        if symbol in trade_summaries and trade_summaries[symbol]:
            summary = trade_summaries[symbol]
            profit_layers_str = ", ".join([
                f"{summary['profit_layers'][i]:.2f} ({'Hit' if summary['profit_layers_hit'][i] else 'Not Hit'})"
                for i in range(NUM_PROFIT_LAYERS)
            ])
            exit_prices_str = ", ".join([f"{p['quantity']} @ {p['price']:.2f}" for p in summary["exit_details"]])
            logger.info(
                f"Trade Summary for {symbol}:\n"
                f"  Type: {summary['trade_type']}\n"
                f"  Entry: {summary['entry_price']:.2f}\n"
                f"  Exits: {exit_prices_str}\n"
                f"  Net P/L: {summary['net_pnl']:.2f}\n"
                f"  Layers: [{profit_layers_str}]\n"
                f"  Reason: {summary['exit_reason']}"
            )
        else:
            logger.info(f"No trade summary available for {symbol}.")

def cleanup(signum=None, frame=None) -> None:
    logger.info("Cleaning up before exit")
    for symbol in SYMBOLS:
        if current_positions[symbol]:
            ltp = ltp_data[symbol]["price"] if ltp_data[symbol]["price"] is not None else current_positions[symbol]["entry_price"]
            qty = current_positions[symbol]["quantity"]
            trade_type = current_positions[symbol]["type"]
            exit_transaction = "SELL" if trade_type == "BUY" else "BUY"
            token, tradingsymbol = TOKEN_MAP.get(symbol, (None, ""))

            if qty > 0 and token and tradingsymbol:
                logger.info(f"Attempting to exit remaining position for {symbol} (Qty: {qty}) due to script shutdown.")
                exit_order_params = {
                    "variety": "NORMAL",
                    "tradingsymbol": tradingsymbol,
                    "symboltoken": str(token),
                    "transactiontype": exit_transaction,
                    "exchange": "NSE",
                    "ordertype": "MARKET",
                    "producttype": "INTRADAY",
                    "duration": "DAY",
                    "price": "0.00",
                    "quantity": str(qty)
                }
                exit_order = place_order(exit_order_params)
                if exit_order and exit_order.get("status"):
                    completed_exit = wait_for_order_completion(exit_order['orderid'])
                    if completed_exit:
                        actual_exit_price = float(completed_exit['averageprice'])
                        fees = qty * actual_exit_price * (BROKERAGE_FEE + STT_CTT + EXCHANGE_FEE + SLIPPAGE)
                        profit = (actual_exit_price - current_positions[symbol]["entry_price"]) * qty if trade_type == "BUY" else (current_positions[symbol]["entry_price"] - actual_exit_price) * qty
                        net_profit = profit - fees
                        trade_summaries[symbol] = {
                            "trade_type": trade_type,
                            "entry_price": current_positions[symbol]["entry_price"],
                            "exit_details": [{"price": actual_exit_price, "quantity": qty}],
                            "net_pnl": net_profit,
                            "profit_layers": [current_positions[symbol]["entry_price"] + d for d in PROFIT_DISTANCES],
                            "profit_layers_hit": partial_profit_taken[symbol].copy(),
                            "exit_reason": "Script Interrupted (Forced Exit)"
                        }
                        logger.info(f"Exited {trade_type} for {symbol}: {qty} at {actual_exit_price:.2f}, Net P/L: {net_profit:.2f}, Reason: Script Interrupted (Forced Exit)")
                    else:
                        logger.error(f"Failed to confirm forced exit order for {symbol}.")
                else:
                    logger.error(f"Failed to place forced exit order for {symbol} on shutdown.")
            else:
                logger.warning(f"No active position or invalid token/tradingsymbol for {symbol} to force exit.")

            if stop_loss_orders[symbol]:
                cancel_order(stop_loss_orders[symbol]["order_id"])
                stop_loss_orders[symbol] = None

            current_positions[symbol] = None

        if websocket_instances[symbol]:
            try:
                if hasattr(websocket_instances[symbol], 'close_connection'):
                    websocket_instances[symbol].close_connection()
                    logger.info(f"Closed WebSocket for {symbol}")
                else:
                    logger.warning(f"No close_connection method for WebSocket {symbol}. Skipping closure.")
            except Exception as e:
                logger.error(f"Error closing WebSocket for {symbol}: {str(e)}")
            websocket_instances[symbol] = None

    log_final_summary()
    logger.info("Cleanup complete")
    sys.exit(0)

# --- Main Execution Block ---
def main() -> None:
    signal.signal(signal.SIGINT, cleanup)
    signal.signal(signal.SIGTERM, cleanup)

    try:
        ist = pytz.timezone('Asia/Kolkata')
        current_dt = datetime.now(ist)
        if not (current_dt.hour >= 9 and (current_dt.hour < 15 or (current_dt.hour == 15 and current_dt.minute < 30))):
            logger.warning("Outside trading hours. Exiting.")
            cleanup()
            return

        for symbol in SYMBOLS:
            start_websocket(symbol)

        threading.Thread(target=websocket_watchdog, daemon=True).start()

        logger.info("Waiting for WebSocket connections and initial LTP...")
        time.sleep(5)

        active_trade_threads = []
        for symbol in SYMBOLS:
            if not current_positions.get(symbol):
                trade_thread = place_manual_trade(symbol, MANUAL_TRADE_TYPE)
                if trade_thread:
                    trade_thread.start()
                    active_trade_threads.append(trade_thread)
                else:
                    logger.error(f"Failed to initiate manual trade for {symbol}.")

        if active_trade_threads:
            logger.info(f"Waiting for {len(active_trade_threads)} trade(s) to complete...")
            for t in active_trade_threads:
                t.join()
            logger.info("All initiated trades have been completed.")
        else:
            logger.info("No trades were initiated successfully.")

    except Exception as e:
        logger.error(f"An error occurred in the main execution block: {str(e)}", exc_info=True)
    finally:
        cleanup()

if __name__ == "__main__":
    main()

[I 250627 12:05:19 ipython-input-7-3589066002:66] Trading script started with fixed target (real trading)
[W 250627 12:05:20 ipython-input-6-1986459775:635] Watchdog: WebSocket for MOTHERSON is inactive. Attempting to restart.
[I 250627 12:05:20 ipython-input-6-1986459775:593] Started WebSocket thread for MOTHERSON
[I 250627 12:05:20 ipython-input-6-1986459775:597] Starting fallback price fetch for MOTHERSON
[E 250627 12:05:20 ipython-input-6-1986459775:600] No token for MOTHERSON in fallback
[D 250627 12:05:20 ipython-input-6-1986459775:554] WebSocket LTP for MOTHERSON: 154.98
[I 250627 12:05:21 ipython-input-6-1986459775:559] WebSocket opened for MOTHERSON
[D 250627 12:05:21 ipython-input-6-1986459775:569] Sending subscription for MOTHERSON: {'correlationID': 'MOTHERSON_1751006120', 'action': 1, 'params': {'mode': 1, 'tokenList': [{'exchangeType': 1, 'tokens': [None]}]}}
[I 250627 12:05:21 ipython-input-6-1986459775:571] Subscribed to MOTHERSON with token None
[D 250627 12:05:22 ipyt

SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
